<a href="https://colab.research.google.com/github/AloysiusMarcello/trial1/blob/main/Monthly_Production_Shift_Utilization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [3]:
!pip install pandas openpyxl

from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
from google.colab import data_table



Mounted at /content/drive


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_rows', None)
from io import StringIO
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import calendar
from datetime import date, datetime, timedelta


# Functions

In [5]:
# def remove_outliers(df, columns, threshold=3):
#     df_clean = df.copy()

#     # Ensure columns are numeric
#     for col in columns:
#         df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

#     # Compute z-scores
#     z_scores = df_clean[columns].apply(zscore)

#     # Keep rows where all z-scores are within threshold
#     df_clean = df_clean[(z_scores.abs() < threshold).all(axis=1)]

#     return df_clean

def remove_outliers(df, columns, threshold=3, max_outlier_cols=1):
    df_clean = df.copy()
    for col in columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

    z_scores = df_clean[columns].apply(zscore)

    # Count how many columns exceed the threshold per row
    outlier_counts = (z_scores.abs() > threshold).sum(axis=1)

    # Keep rows with fewer than or equal to max_outlier_cols
    df_clean = df_clean[outlier_counts <= max_outlier_cols]

    return df_clean

def remove_outliersR(df, columns, threshold=3):
    df_clean = df.copy()

    # Ensure columns are numeric
    for col in columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

        # Compute z-scores for the column
        z = zscore(df_clean[col], nan_policy='omit')

        # Mask outliers only in this column
        df_clean.loc[z > threshold, col] = np.nan
        df_clean.loc[z < -threshold, col] = np.nan

    return df_clean

def remove_outliersIQR(df, columns, k=1.5):
    df_clean = df.copy()

    for col in columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - k * IQR
        upper = Q3 + k * IQR
        df_clean.loc[(df_clean[col] < lower) | (df_clean[col] > upper), col] = np.nan

    return df_clean

plt.style.use('default')

def plot2(df, x, y_listTemp, y_listHum, title):

    fig, axs = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    fig.suptitle(title, fontsize=16)

    # Plot Temperature(s)
    for y in y_listTemp:
        axs[0].plot(df[x], df[y], label=y)
    axs[0].set_ylabel('Temperature (°C)')
    axs[0].legend()
    axs[0].grid(True)

    # Plot Humidity(s)
    for y in y_listHum:
        axs[1].plot(df[x], df[y], label=y)
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Humidity (%RH)')
    axs[1].legend()
    axs[1].grid(True)

    # Adjust layout and display the figure just once
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

def plotme(df, x, y_list, title, xlabel, ylabel):
    df = df.copy()
    df.columns = df.columns.str.strip()
    df[x] = pd.to_datetime(df[x], errors='coerce')

    plt.figure(figsize=(14, 5))

    for y in y_list:
        df[y] = pd.to_numeric(df[y], errors='coerce')

        # Filter out rows where y is NaN or 0, but keep matching x
        filtered = df[df[y].notna() & (df[y] != 0)]
        plt.plot(filtered[x], filtered[y], label=y)
        #plt.plot(filtered[x], filtered[y], color='blue', marker='o')

    plt.style.use('default')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plotbig(df, x, y_list, title, xlabel, ylabel):
    df = df.copy()
    df.columns = df.columns.str.strip()
    df[x] = pd.to_datetime(df[x], errors='coerce')

    plt.figure(figsize=(14, 10))

    for y in y_list:
        df[y] = pd.to_numeric(df[y], errors='coerce')

        # Filter out rows where y is NaN or 0, but keep matching x
        filtered = df[df[y].notna() & (df[y] != 0)]
        plt.plot(filtered[x], filtered[y], label=y)
        #plt.plot(filtered[x], filtered[y], color='blue', marker='o')

    plt.style.use('default')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


def boxplotbig(df, time_col, value_cols, title, xlabel, ylabel):
    """
    Plots a boxplot for selected columns in a dataframe.

    Parameters:
    - df: pandas DataFrame
    - time_col: str, name of the timestamp column (not used directly but kept for consistency)
    - value_cols: list of str, column names to plot
    - title: str, plot title
    - xlabel: str, label for x-axis
    - ylabel: str, label for y-axis
    """
    # Melt the DataFrame into long format
    df_melted = df[value_cols].melt(var_name='Variable', value_name='Value')

    # Plot
    plt.figure(figsize=(16, 10))
    sns.boxplot(data=df_melted, x='Variable', y='Value')
    plt.xticks(rotation=45)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def count_weekdays(year: int, month_str: str):
    # Convert month name to month number
    month_str = month_str.capitalize()[:3]  # e.g., "jan", "JANUARY" → "Jan"
    month = list(calendar.month_abbr).index(month_str)

    # Get the first and last day of the month
    start_date = f'{year}-{month:02d}-01'
    end_day = calendar.monthrange(year, month)[1]
    end_date = f'{year}-{month:02d}-{end_day}'

    # Generate date range
    dates = pd.date_range(start=start_date, end=end_date)

    # Count weekdays (Mon–Fri)
    return sum(d.weekday() < 5 for d in dates)

def count_total_days(year: int, month_str: str):
    # Normalize the month string
    month_str = month_str.capitalize()[:3]  # e.g., "january" → "Jan"
    month = list(calendar.month_abbr).index(month_str)

    # Get number of days in the month
    total_days = calendar.monthrange(year, month)[1]
    return total_days

def count_total_days_excluding_sundays(year: int, month_str: str):
    # Normalize the month string
    month_str = month_str.capitalize()[:3]
    month = list(calendar.month_abbr).index(month_str)

    # Get number of days in the month
    total_days = calendar.monthrange(year, month)[1]

    # Count days that are not Sunday
    count = 0
    for day in range(1, total_days + 1):
        if date(year, month, day).weekday() != 6:  # 6 = Sunday
            count += 1
    return count


# Analyze SABRE Runtime

In [6]:
file_path = '/content/drive/MyDrive/NUS/Ferron_Data_Analyst_Intern/Production_Utilization_Case/Data_SABRE.xlsx'

df = pd.read_excel(file_path)
df = df.drop_duplicates()
df = df[df['LINI'] == 'LIQUID']
df = df[df['RESOURCES DESC'] == 'MACHINE FILLING MARCHESINI ML616 LIQUID']
#df = df[df['MFG PRODUCT DESC'] == 'CETIRIZINE HYDROCHLORIDE SYRUP 5 MG/5 ML 60 ML']
#data_table.DataTable(df)


In [7]:
print(df['MFG PRODUCT DESC'].nunique())

31


In [8]:
# Example: Detect outliers in a 'Value' column
mean = df['LT SFO FPP'].mean()
print('mean:', mean)
std = df['LT SFO FPP'].std()
print('std:', std)

# Z-score calculation
df['z_score'] = (df['LT SFO FPP'] - mean) / std

# Flag outliers
outliers = df[np.abs(df['z_score']) > 3]


mean: 25.333420245397544
std: 34.02377975698596


In [9]:
outliers.head()

,RESOURCES,RESOURCES DESC,LINI,MFG PRODUCT ID,MFG PRODUCT DESC,BATCH NO,SFO START DATE,SFO END DATE,OUTPUT,LT SFO FPP,TGT PRODUCTIVITY DET,ACT PRODUCTIVITY DET,A/T,z_score
749,FPMCFILLMARML6,MACHINE FILLING MARCHESINI ML616 LIQUID,LIQUID,A-10275-00,VALPROIC ACID SYRUP 250 MG / 5 ML,5242312A,2025-01-17 02:11:15,2025-01-30 15:26:38,27643.0,325.256389,401.0,84.988338,0.211941,8.815098
824,FPMCFILLMARML6,MACHINE FILLING MARCHESINI ML616 LIQUID,LIQUID,A-14593-00,RHINOS NEO DROPS 10 ML,4150007,2025-01-28 14:25:24,2025-02-03 12:37:36,55985.0,142.203333,401.0,393.696819,0.981788,3.434948


In [10]:
#OUTLIERS

# Function to flag outliers in each group
def detect_outliers(group):
    Q1 = group['LT SFO FPP'].quantile(0.25)
    Q3 = group['LT SFO FPP'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return group[(group['LT SFO FPP'] < lower) | (group['LT SFO FPP'] > upper)]

# Apply to each product group
outliers_df = df.groupby('MFG PRODUCT ID').apply(detect_outliers).reset_index(drop=True)



# Removing the Outliers
df = df.merge(outliers_df, how='outer', indicator=True).query('_merge == "left_only"').drop(columns=['_merge'])

# Merging the product's SFO into one average SFO
df = df.groupby('MFG PRODUCT ID', as_index=False)['LT SFO FPP'].mean()
df = df.rename(columns={'LT SFO FPP': 'Run Filling',
                        'MFG PRODUCT ID': 'Item Number'})
data_table.DataTable(df)


/tmp/ipython-input-10-3340221264.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outliers_df = df.groupby('MFG PRODUCT ID').apply(detect_outliers).reset_index(drop=True)


,Item Number,Run Filling
0,A-10021-00,7.408889
1,A-10056-00,25.354167
2,A-10113-00,8.063958
3,A-10145-00,43.909167
4,A-10275-00,17.524839
5,A-10276-00,79.965278
6,A-10495-00,13.736944
7,A-10627-00,20.679097
8,A-10883-00,22.236204
9,A-10939-00,15.063722


# Standardized Runtime

In [11]:
file_path = '/content/drive/MyDrive/NUS/Ferron_Data_Analyst_Intern/Production_Utilization_Case/File_Pak_Rama.xlsx'

dfR = pd.read_excel(file_path)
dfR = dfR.drop_duplicates()
dfR = dfR.drop(columns = ['Man','Batch Size', 'ManHour', 'Man.1','ManHour.1', 'Man.2', 'ManHour.2', 'Run Blowing', 'Man.3', 'ManHour.3', 'Man.4', 'ManHour.4','Set Up Blowing'])
dfR = dfR.drop(dfR.columns[6:41], axis = 1)
dfR = dfR.dropna()

# Removing trailing spaces/'\n's
dfR['Item Number'] = dfR['Item Number'].astype(str).str.strip()

#data_table.DataTable(dfR)
dfR.tail(10)

,Item Number,Description,SuCu Mixing (Include CIP),Run Mixing,SuCu Filling,Run Filling
84,A-19258-00,VOMETA SUSPENSION 5 MG/5 ML 60 ML SRILANKA,5,6.0,6,2.710000
85,A-19288-00,STIMUNO SYRUP 25 MG/5 ML 100 ML CAMBODIA,5,10.0,6,3.921667
86,A-19393-00,VOMETA SUSPENSION 5 MG/ 5 ML 60 ML PHILIPPINES,5,6.0,6,16.250000
87,A-19397-00,COLERGIS SYRUP 60 ML VIETNAM,5,2.0,5,3.252000
88,A-19398-00,VOMETA DROPS 5 MG/ML 10 ML PHILIPPINES,5,3.0,5,3.174667
89,A-12312-00,Lactulose Syrup 3.3 G/ 5ml 60 ml,2,0.0,6,4.531000
90,A-12705-00,Valtor Syrup 250 mg / 5 ml 100 ml Philippines,5,5.0,5,9.313667
91,A-12473-00,Imuncare Syrup 25 mg/5 ml 60 ml Philippines,5,10.0,6,16.260000
92,A-12472-00,Imuncare Syrup 25 mg/5 ml 100 ml Philippines,5,10.0,6,9.803667
93,A-12771-00,Psidii Syrup 250 mg/5 ml 60 ml Cambodia,5,8.0,5,3.252000


In [12]:
# dffin.to_excel('Final Runtime SABRE.xlsx', index=False)
# files.download('Final Runtime SABRE.xlsx')

# PPIC Master Schedule

## Dataset Read

In [164]:
filename = 'MS Jul.xlsx'
month = filename[3:6]
year = '2025'
shiftNo = '2 Shifts'
# Overtime Options: 'Weekday', 'Weekday + Saturday', '7 Days'
useOvertime = 'Weekday + Saturday'

HoldingTank = [
    ('A-14593-00', 'Rhinos Neo Drops 10 ml'),
    ('A-14593-00', 'Rhinos Neo Drops 10 ml'),
    ('', '')
]

sheetname = 'Likuida'

### Data Prep

In [165]:

file_path = '/content/drive/MyDrive/NUS/Ferron_Data_Analyst_Intern/Production_Utilization_Case/' + filename

dfMS = pd.read_excel(file_path, sheet_name=sheetname, usecols=[3, 4, 11, 15], skiprows = 8, header = 0)

# column rename
dfMS.columns = ['Item Number', 'Batch', 'Idx Nas', 'Due Date']

# fluran = False
# for i in range(len(dfMS)-1):
#   if type(dfMS['Batch'][i]) == 'string':
#     if ('Inhalation' in dfMS['Batch'][i]) and ('Inhalation' not in dfMS['Batch'][i+1]):
#       print(dfMS['Batch'][i])
#       dfMS = dfMS[i+4:]

dfMS = dfMS[~dfMS['Batch'].astype(str).str.contains('Inhalation', na=False)]


# to datetime
dfMS['Due Date'] = pd.to_datetime(dfMS['Due Date'], dayfirst = False)

# remove NAs
dfMS = dfMS.dropna()

# Removing trailing spaces/'\n's
dfMS['Item Number'] = dfMS['Item Number'].astype(str).str.strip()

# Reseting Index
dfMS = dfMS.reset_index(drop=True)


#dfMS.head()
data_table.DataTable(dfMS)


,Item Number,Batch,Idx Nas,Due Date
0,A-10962-00,Neuvis Pro Syrup 500 mg/5 ml 60 ml,4.15627,2025-07-30
1,A-19155-00,Vometa Drops 5 mg/ml 10 ml Myanmar,4.215603,2025-07-30
2,A-19155-00,Vometa Drops 5 mg/ml 10 ml Myanmar,4.215603,2025-07-30
3,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28
4,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28
5,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28
6,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28
7,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28
8,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28
9,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28


## Item - Bulk List

In [166]:
file_path = '/content/drive/MyDrive/NUS/Ferron_Data_Analyst_Intern/Production_Utilization_Case/Item Bulk List.xlsx'

dfblist = pd.read_excel(file_path)

bdict = dfblist.set_index('Product No')['Ingredient No'].to_dict()

#dfblist.head()
print(bdict)


{'A-10048-00': 'B-10453-00', 'A-10939-00': 'B-10453-00', 'A-14547-00': 'B-10453-00', 'A-19209-00': 'B-10445-00', 'A-19397-00': 'B-10445-00', 'A-10495-00': 'B-10345-00', 'A-11045-00': 'B-10345-00', 'A-19393-00': 'B-10345-00', 'A-10641-00': 'B-10263-00', 'A-10861-00': 'B-10263-00', 'A-12231-00': 'B-10263-00', 'A-12312-00': 'B-10411-00', 'A-10113-00': 'B-10435-00', 'A-10145-00': 'B-10435-00', 'A-17029-00': 'B-10435-00', 'A-10021-00': 'B-10418-00', 'A-12298-00': 'B-10418-00', 'A-12709-00': 'B-10503-00', 'A-12628-00': 'B-10503-00', 'A-10038-00': 'B-10264-00', 'A-10056-00': 'B-10264-00', 'A-10137-00': 'B-10264-00', 'A-10627-00': 'B-10264-00', 'A-12472-00': 'B-10264-00', 'A-12473-00': 'B-10264-00', 'A-19288-00': 'B-10264-00', 'A-10275-00': 'B-10434-00', 'A-10276-00': 'B-10434-00', 'A-10597-00': 'B-10434-00', 'A-12239-00': 'B-10434-00', 'A-12315-00': 'B-10434-00', 'A-12414-00': 'B-10434-00', 'A-12705-00': 'B-10434-00'}


## Bulk Batches


In [167]:
bulk = False
curr = ''
cbulk = ''
dfMS['Bulk'] = False

for i in range(len(dfMS)):
  if dfMS['Item Number'][i][0] == 'A':
    #dfMS.loc[i, 'Bulk']= bulk
    # if 1 < i+1 < len(dfMS) and dfMS['Item Number'][i-1][0] == 'B' and dfMS['Item Number'][i+1][0] == 'B':
    #   dfMS.loc[i, 'Bulk']= False
    #   bulk = False
    if (dfMS['Item Number'][i] in bdict) and (bdict[dfMS['Item Number'][i]] == cbulk):
      #print(dfMS.loc[i, 'Bulk'], 'True')
      dfMS.loc[i, 'Bulk']= True
      #bulk = True
    # if i+1 < len(dfMS) and dfMS['Item Number'][i+1][0] == 'A' and dfMS['Item Number'][i+1] != dfMS['Item Number'][i]:
    #   dfMS.loc[i, 'Bulk']= False
    #   bulk = False
    # else:
    #   print(dfMS['Item Number'][i], bulk)
    #   dfMS.loc[i, 'Bulk']= bulk
  else:
    #print("happen", i)
    dfMS.loc[i, 'Bulk'] = 'bulk'
    cbulk = dfMS['Item Number'][i]
    #print(cbulk)
    #bulk = True

data_table.DataTable(dfMS)
#dfMS.head(10)

/tmp/ipython-input-167-352993775.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'bulk' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  dfMS.loc[i, 'Bulk'] = 'bulk'


,Item Number,Batch,Idx Nas,Due Date,Bulk
0,A-10962-00,Neuvis Pro Syrup 500 mg/5 ml 60 ml,4.15627,2025-07-30,False
1,A-19155-00,Vometa Drops 5 mg/ml 10 ml Myanmar,4.215603,2025-07-30,False
2,A-19155-00,Vometa Drops 5 mg/ml 10 ml Myanmar,4.215603,2025-07-30,False
3,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28,bulk
4,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28,True
5,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28,bulk
6,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28,True
7,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28,bulk
8,A-10021-00,Psidii Syrup 250 mg /5 ml 60 ml,4.62065,2025-08-28,True
9,B-10418-00,Bulk Psidii Syrup 250 mg /5 ml,noidx,2025-08-28,bulk


### Fail(s) :)

In [168]:
# bulk = False
# curr = ''
# cbulk = ''

# for i in range(len(dfMS)):
#   if dfMS['Item Number'][i][0] == 'A':
#     dfMS.loc[i, 'Bulk']= bulk
#     # if 1 < i+1 < len(dfMS) and dfMS['Item Number'][i-1][0] == 'B' and dfMS['Item Number'][i+1][0] == 'B':
#     #   dfMS.loc[i, 'Bulk']= False
#     #   bulk = False
#     if bdict[dfMS['Item Number'][i]] == cbulk:
#       dfMS.loc[i, 'Bulk']= True
#       bulk = True
#     if i+1 < len(dfMS) and dfMS['Item Number'][i+1][0] == 'A' and dfMS['Item Number'][i+1] != dfMS['Item Number'][i]:
#       dfMS.loc[i, 'Bulk']= False
#       bulk = False
#     # else:
#     #   print(dfMS['Item Number'][i], bulk)
#     #   dfMS.loc[i, 'Bulk']= bulk
#   else:
#     #print("happen", i)
#     dfMS.loc[i, 'Bulk'] = 'bulk'
#     cbulk = dfMS['Item Number'][i]
#     bulk = True

# #data_table.DataTable(dfMS)
# dfMS.head(60)

In [169]:
# bulk = False
# curr = ''
# for i in range(len(dfMS)):
#   if dfMS['Item Number'][i][0] == 'A':
#     if dfMS.loc[i, 'Item Number'] != curr:
#       bulk = False
#       curr = dfMS['Item Number'][i]
#     elif i+1 < len(dfMS) and dfMS['Item Number'][i+1][0] != dfMS['Item Number'][i]:
#       dfMS.loc[i, 'Bulk']= False
#     else:
#       print(dfMS['Item Number'][i], bulk)
#       dfMS.loc[i, 'Bulk']= bulk

#   else:
#     dfMS.loc[i, 'Bulk'] = 'bulk'
#     bulk = True
#     curr = dfMS['Item Number'][i+1]

# data_table.DataTable(dfMS)
# #dfMS.head(20)

# Runtime Integration

### Runtime

In [170]:
# Uppercase 'Batch'
dfMS['Batch'] = dfMS['Batch'].str.upper()

# Copy for rerunning
dfSFO = dfMS.copy()

# Insert Mixing Runtime
dfSFO['SuCu Mixing'] = 0.0 # To be added
dfSFO['Mixing Runtime'] = 0.0
dfSFO['SuCu Filling'] = 0.0
dfSFO['Filling Runtime'] = 0.0

for i in range(len(dfSFO)):
  # matching_row = dfR[dfR['Item Number'] == dfSFO.loc[i, 'Item Number']]
  # if not matching_row.empty:
    if dfSFO.loc[i, 'Bulk'] == False: # if False
      print(dfSFO['Item Number'][i], dfSFO['Batch'][i], 'False')
      dfSFO.loc[i, 'SuCu Mixing'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Mixing (Include CIP)'].values[0]  # matching_row['SuCu Mixing (Include CIP)'].values[0] # To be added
      dfSFO.loc[i, 'Mixing Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Mixing'].values[0]
      dfSFO.loc[i, 'SuCu Filling'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Filling'].values[0]
      dfSFO.loc[i, 'Filling Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Filling'].values[0]
    elif dfSFO.loc[i, 'Bulk'] == 'bulk': # If Bulk
      print(dfSFO['Item Number'][i], dfSFO['Batch'][i], 'Bulk')
      dfSFO.loc[i, 'SuCu Mixing'] =  dfR.loc[dfR['Item Number'] == (dfblist.loc[dfblist['Ingredient No'] == dfSFO['Item Number'][i], 'Product No'].values[0]), 'SuCu Mixing (Include CIP)'].values[0]
      dfSFO.loc[i, 'Mixing Runtime'] = dfR.loc[dfR['Item Number'] == (dfblist.loc[dfblist['Ingredient No'] == dfSFO['Item Number'][i], 'Product No'].values[0]), 'Run Mixing'].values[0]

    elif dfSFO.loc[i,'Bulk'] == True: # If True
      print(dfSFO['Item Number'][i], dfSFO['Batch'][i], 'True')
      dfSFO.loc[i, 'SuCu Filling'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Filling'].values[0]
      dfSFO.loc[i, 'Filling Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Filling'].values[0]


data_table.DataTable(dfSFO,)
#dfSFO.head(20)

A-10962-00 NEUVIS PRO SYRUP 500 MG/5 ML 60 ML False
A-19155-00 VOMETA DROPS 5 MG/ML 10 ML MYANMAR False
A-19155-00 VOMETA DROPS 5 MG/ML 10 ML MYANMAR False
B-10418-00 BULK PSIDII SYRUP 250 MG /5 ML Bulk
A-10021-00 PSIDII SYRUP 250 MG /5 ML 60 ML True
B-10418-00 BULK PSIDII SYRUP 250 MG /5 ML Bulk
A-10021-00 PSIDII SYRUP 250 MG /5 ML 60 ML True
B-10418-00 BULK PSIDII SYRUP 250 MG /5 ML Bulk
A-10021-00 PSIDII SYRUP 250 MG /5 ML 60 ML True
B-10418-00 BULK PSIDII SYRUP 250 MG /5 ML Bulk
A-12298-00 THROMBOLIEF SYRUP 250 MG/5 ML 60 ML PHILIPPINES True
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
A-12312-00 LACTULOSE SYRUP 3.3 G/ 5ML 60 ML False
B-10434-00 BULK VALPROIC SYRUP 250 MG / 5 ML Bulk
A-10275-00 VALPROIC SYRUP 250 MG / 5 ML True
B-10434-00 BULK VALPROIC SYRUP 250 MG / 5 ML Bulk
A-1

,Item Number,Batch,Idx Nas,Due Date,Bulk,SuCu Mixing,Mixing Runtime,SuCu Filling,Filling Runtime
0,A-10962-00,NEUVIS PRO SYRUP 500 MG/5 ML 60 ML,4.15627,2025-07-30,False,5.0,10.0,5.0,9.803667
1,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667
2,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667
3,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
4,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000
5,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
6,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000
7,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
8,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000
9,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000


#### Failed

In [171]:
# # Insert Mixing Runtime
# dfSFO['SuCu Mixing'] = 0.0 # To be added
# dfSFO['Mixing Runtime'] = 0.0
# dfSFO['SuCu Filling'] = 0.0
# dfSFO['Filling Runtime'] = 0.0

# for i in range(len(dfSFO)):
#   # matching_row = dfR[dfR['Item Number'] == dfSFO.loc[i, 'Item Number']]
#   # if not matching_row.empty:
#     if dfSFO.loc[i, 'Bulk'] == False: # if False
#       print(dfSFO['Batch'][i], 'False')
#       dfSFO.loc[i, 'SuCu Mixing'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Mixing (Include CIP)'].values[0]  # matching_row['SuCu Mixing (Include CIP)'].values[0] # To be added
#       dfSFO.loc[i, 'Mixing Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Mixing'].values[0]
#       dfSFO.loc[i, 'SuCu Filling'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Filling'].values[0]
#       dfSFO.loc[i, 'Filling Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Filling'].values[0]
#     # elif dfSFO.loc[i, 'Bulk'] == 'bulk': # If Bulk
#     #   print(dfSFO['Batch'][i], 'Bulk')
#     #   dfSFO.loc[i, 'SuCu Mixing'] = dfSFO.loc[i, 'SuCu Mixing'] = dfR.loc[dfR['Description'].str.contains(str(dfSFO['Batch'].iloc[i])[5:], na=False), 'SuCu Mixing (Include CIP)'].values[0]
#     #   dfSFO.loc[i, 'Mixing Runtime'] = dfSFO.loc[i, 'Mixing Runtime'] = dfR.loc[dfR['Description'].str.contains(str(dfSFO['Batch'].iloc[i])[5:], na=False), 'Run Mixing'].values[0]

#     elif dfSFO.loc[i,'Bulk'] == True: # If True
#       print(dfSFO['Batch'][i], 'True')
#       dfSFO.loc[i, 'SuCu Filling'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'SuCu Filling'].values[0]
#       dfSFO.loc[i, 'Filling Runtime'] = dfR.loc[dfR['Item Number'] == dfSFO['Item Number'][i], 'Run Filling'].values[0]


# data_table.DataTable(dfSFO)

### Campaign Runtime

#### SuCu Filling

In [172]:

camp = 0
#campB = 0
curr = ''
for i in range(len(dfSFO)):
  if dfSFO['Bulk'][i] != 'bulk' :
    if dfSFO.loc[i, 'Item Number'] != curr:
      camp = 0
      curr = dfSFO['Item Number'][i]
    elif camp < 2:
      camp += 1
      dfSFO.loc[i, 'SuCu Filling'] = 1
    else:
      camp = 0
      #dfSFO.loc[i, 'SuCu Filling'] =
  # else:
  #   camp = 0
  #   curr = ''


data_table.DataTable(dfSFO)
#dfSFO.head(20)




,Item Number,Batch,Idx Nas,Due Date,Bulk,SuCu Mixing,Mixing Runtime,SuCu Filling,Filling Runtime
0,A-10962-00,NEUVIS PRO SYRUP 500 MG/5 ML 60 ML,4.15627,2025-07-30,False,5.0,10.0,5.0,9.803667
1,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667
2,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,1.0,3.174667
3,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
4,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000
5,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
6,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000
7,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
8,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000
9,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000


#### SuCu Mixing

In [173]:
camp = 0
#campB = 0
curr = ''
for i in range(len(dfSFO)):
  if dfSFO['Bulk'][i] != True :
    if dfSFO.loc[i, 'Item Number'] != curr:
      camp = 0
      curr = dfSFO['Item Number'][i]
    # elif camp < 2:
    #   camp += 1
    #   dfSFO.loc[i, 'SuCu Filling'] = 1
    else:
      camp = 0
      dfSFO.loc[i, 'SuCu Mixing'] = 3
      #dfSFO.loc[i, 'SuCu Filling'] =



data_table.DataTable(dfSFO)
#dfSFO.head(20)




,Item Number,Batch,Idx Nas,Due Date,Bulk,SuCu Mixing,Mixing Runtime,SuCu Filling,Filling Runtime
0,A-10962-00,NEUVIS PRO SYRUP 500 MG/5 ML 60 ML,4.15627,2025-07-30,False,5.0,10.0,5.0,9.803667
1,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667
2,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,3.0,3.0,1.0,3.174667
3,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000
4,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000
5,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000
6,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000
7,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000
8,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000
9,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000


# Shift Utilization

In [174]:
dfUtil = dfSFO.copy()
dfUtil['Total Mixing Runtime'] = dfUtil['SuCu Mixing'] + dfUtil['Mixing Runtime']
dfUtil['Total Filling Runtime'] = dfUtil['SuCu Filling'] + dfUtil['Filling Runtime']

sumMix = dfUtil['Total Mixing Runtime'].sum()/8
sumFil = dfUtil['Total Filling Runtime'].sum()/8

totalweekdays = count_weekdays(2025, month)
print(f'Total Weekdays: {totalweekdays}')
monthshift5 = totalweekdays*2

totaldays = count_total_days(2025, month)
print(f'Total Days: {totaldays}')
monthshift7 = totaldays*2

total6days = count_total_days_excluding_sundays(2025, month)
print(f'Total 6 Days: {total6days}')
monthshift6 = total6days*2





with open("Shift Utilization.txt", "w") as f:
    f.write(f"Sum of Total Mixing Runtime: {sumMix} shifts\n")
    f.write(f"5 Days of shift Utilization: {round(sumMix/monthshift5*100, 2)}%\n")
    f.write(f"6 Days of shift Utilization: {round(sumMix/monthshift6*100, 2)}%\n")
    f.write(f"7 Days of shift Utilization: {round(sumMix/monthshift7*100, 2)}%\n\n")

    f.write(f"Sum of Total Filling Runtime: {sumFil} shifts\n")
    f.write(f"5 Days of shift Utilization: {round(sumFil/monthshift5*100, 2)}%\n")
    f.write(f"6 Days of shift Utilization: {round(sumFil/monthshift6*100, 2)}%\n")
    f.write(f"7 Days of shift Utilization: {round(sumFil/monthshift7*100, 2)}%\n")

files.download("Shift Utilization.txt")


Total Weekdays: 23
Total Days: 31
Total 6 Days: 27


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SCHEDULING

### Overtime & Feasability

In [175]:
overtime = True if (sumMix/monthshift5*100 > 100) or (sumFil/monthshift5*100 > 100) else False
feasible = False if (sumMix/monthshift7*100 > 100) or (sumFil/monthshift7*100 > 100) else True

print(f'Overtime: {overtime}')
if (sumMix/monthshift5*100 > 100):
  print(f'Needs {sumMix*8-monthshift5*8} more mixing hours')
if (sumFil/monthshift5*100 > 100):
  print(f'Needs {sumFil*8-monthshift5*8} more filling hours')
print(f'Feasible: {feasible}')
if (sumMix/monthshift7*100 > 100):
  print(f'Leak {sumMix*8-monthshift7*8} more mixing hours')
if (sumFil/monthshift7*100 > 100):

  print(f'Leak {sumFil*8-monthshift7*8} more filling hours')

data_table.DataTable(dfUtil)
#dfUtil.head(20)

# dfUtil.to_csv("Utilization.csv", index=False)
# files.download("Utilization.csv")

Overtime: False
Feasible: True


,Item Number,Batch,Idx Nas,Due Date,Bulk,SuCu Mixing,Mixing Runtime,SuCu Filling,Filling Runtime,Total Mixing Runtime,Total Filling Runtime
0,A-10962-00,NEUVIS PRO SYRUP 500 MG/5 ML 60 ML,4.15627,2025-07-30,False,5.0,10.0,5.0,9.803667,15.0,14.803667
1,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667,8.0,8.174667
2,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,3.0,3.0,1.0,3.174667,6.0,4.174667
3,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,5.0,8.0,0.0,0.000000,13.0,0.000000
4,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,5.0,3.252000,0.0,8.252000
5,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000,11.0,0.000000
6,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000,0.0,4.252000
7,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000,11.0,0.000000
8,A-10021-00,PSIDII SYRUP 250 MG /5 ML 60 ML,4.62065,2025-08-28,True,0.0,0.0,1.0,3.252000,0.0,4.252000
9,B-10418-00,BULK PSIDII SYRUP 250 MG /5 ML,noidx,2025-08-28,bulk,3.0,8.0,0.0,0.000000,11.0,0.000000


## Campaign Labelling

In [176]:
dfCamp = dfUtil.copy()

curB = ''
curA = ''
countB = 1
countA = 1
for i in range(len(dfCamp)):
  if dfCamp['Bulk'][i] != 'bulk':
    if dfCamp['Batch'][i] != curA:
      curA = dfCamp.loc[i,'Batch']
      countA = 1
    else:
      countA += 1
      dfCamp.loc[i,'Batch'] = f"{dfCamp.loc[i,'Batch']} ({countA})"
  else:
    if dfCamp['Batch'][i] != curB:
      curB = dfCamp.loc[i,'Batch']
      countB = 1
    else:
      countB += 1
      dfCamp.loc[i,'Batch'] = f"{dfCamp.loc[i,'Batch']} ({countB})"

# data_table.DataTable(dfCamp)

In [177]:
dfUtil = dfCamp.copy()

## Index Sorting

### Failed 1

In [178]:


# dfSort2 = dfUtil.copy()

# # Step 1: Clean types
# dfSort2['Idx Nas'] = pd.to_numeric(dfSort2['Idx Nas'], errors='coerce').fillna(0)
# dfSort2['Due Date'] = pd.to_datetime(dfSort2['Due Date'])
# dfSort2['OriginalIndex'] = dfSort2.index

# # Step 2: Extract base product name for matching B-A pairs
# def extract_base(batch):
#     return str(batch).split('(')[0].strip().replace("BULK", "").strip()

# dfSort2['BaseName'] = dfSort2['Batch'].apply(extract_base)

# # Step 3: Group by BaseName + Due Date
# # (Due Date included in case similar names exist across different schedules)
# group_keys = dfSort2.groupby(['BaseName', 'Due Date']).ngroup()

# dfSort2['CampaignGroup'] = group_keys

# # Step 4: Identify if a group is a campaign (contains any bulk or 'bulk')
# dfSort2['IsCampaign'] = dfSort2['Bulk'].apply(lambda x: x in ['bulk', True])

# group_campaign_flag = dfSort2.groupby('CampaignGroup')['IsCampaign'].any()
# dfSort2['IsCampaign'] = dfSort2['CampaignGroup'].map(group_campaign_flag)

# # Step 5: Create sorting keys for each CampaignGroup
# group_due_dates = dfSort2.groupby('CampaignGroup')['Due Date'].min()
# min_idxs = dfSort2[~dfSort2['IsCampaign']].groupby('CampaignGroup')['Idx Nas'].min()

# sort_keys = pd.concat([group_due_dates, min_idxs], axis=1)
# sort_keys.columns = ['GroupDueDate', 'Idx Nas']
# sort_keys['Idx Nas'] = sort_keys['Idx Nas'].fillna(float('inf'))
# sort_keys = sort_keys.sort_values(['GroupDueDate', 'Idx Nas']).reset_index()

# # Step 6: Map GroupOrder
# group_order = {g: i for i, g in enumerate(sort_keys['CampaignGroup'])}
# dfSort2['GroupOrder'] = dfSort2['CampaignGroup'].map(group_order)

# # Step 7: Final sorting, preserving original internal order (B-A-B-A)
# dfSorted2 = (
#     dfSort2.sort_values(['GroupOrder', 'OriginalIndex'])
#     #.drop(columns=['CampaignGroup', 'GroupOrder', 'IsCampaign', 'OriginalIndex'])
#     .reset_index(drop=True)
# )

# # ✅ Ready for display or scheduling
# data_table.DataTable(dfSorted2)


### Success

In [179]:
dfSort2 = dfUtil.copy()

# Step 1: Clean types
dfSort2['Idx Nas'] = pd.to_numeric(dfSort2['Idx Nas'], errors='coerce').fillna(0)
dfSort2['Due Date'] = pd.to_datetime(dfSort2['Due Date'])
dfSort2['OriginalIndex'] = dfSort2.index

# Step 2: Extract base name for grouping
def extract_base(batch):
    initial = str(batch).split('(')[0].strip().replace("BULK", "").strip()
    return initial[:15]

dfSort2['BaseName'] = dfSort2['Batch'].apply(extract_base)

# Step 3: Group by BaseName ONLY — to keep all A/B pairs of same product together
dfSort2['CampaignGroup'] = dfSort2.groupby('BaseName', sort=False).ngroup()

# Step 4: Mark Campaigns if there's any Bulk in the group
dfSort2['IsCampaign'] = dfSort2['Bulk'].apply(lambda x: x in ['bulk', True])
group_flags = dfSort2.groupby('CampaignGroup')['IsCampaign'].any()
dfSort2['IsCampaign'] = dfSort2['CampaignGroup'].map(group_flags)

# Step 5: Sort CampaignGroups by Due Date and Idx Nas of NON-Bulk (for priority)
group_due = dfSort2.groupby('CampaignGroup')['Due Date'].min()
group_idx = dfSort2[~dfSort2['IsCampaign']].groupby('CampaignGroup')['Idx Nas'].min()

sort_keys = pd.concat([group_due, group_idx], axis=1)
sort_keys.columns = ['GroupDueDate', 'Idx Nas']
sort_keys['Idx Nas'] = sort_keys['Idx Nas'].fillna(float('inf'))
sort_keys = sort_keys.sort_values(['GroupDueDate', 'Idx Nas']).reset_index()

# Step 6: Map to GroupOrder
group_order = {g: i for i, g in enumerate(sort_keys['CampaignGroup'])}
dfSort2['GroupOrder'] = dfSort2['CampaignGroup'].map(group_order)

# Step 7: Sort by GroupOrder and OriginalIndex (to preserve B-A-B-A structure)
dfSorted2 = (
    dfSort2.sort_values(['GroupOrder', 'OriginalIndex'])
    .reset_index(drop=True)
)

# Optional: Display or continue processing
data_table.DataTable(dfSorted2)


,Item Number,Batch,Idx Nas,Due Date,Bulk,SuCu Mixing,Mixing Runtime,SuCu Filling,Filling Runtime,Total Mixing Runtime,Total Filling Runtime,OriginalIndex,BaseName,CampaignGroup,IsCampaign,GroupOrder
0,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,3.240155,2025-07-30,False,2.0,0.0,6.0,4.531000,2.0,10.531000,11,LACTULOSE SYRUP,4,False,0
1,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),3.240155,2025-07-30,False,3.0,0.0,1.0,4.531000,3.0,5.531000,12,LACTULOSE SYRUP,4,False,0
2,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (3),3.240155,2025-07-30,False,3.0,0.0,1.0,4.531000,3.0,5.531000,13,LACTULOSE SYRUP,4,False,0
3,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (4),3.240155,2025-07-30,False,3.0,0.0,6.0,4.531000,3.0,10.531000,14,LACTULOSE SYRUP,4,False,0
4,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (5),3.240155,2025-07-30,False,3.0,0.0,1.0,4.531000,3.0,5.531000,15,LACTULOSE SYRUP,4,False,0
5,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (6),3.240155,2025-07-30,False,3.0,0.0,1.0,4.531000,3.0,5.531000,16,LACTULOSE SYRUP,4,False,0
6,A-10962-00,NEUVIS PRO SYRUP 500 MG/5 ML 60 ML,4.156270,2025-07-30,False,5.0,10.0,5.0,9.803667,15.0,14.803667,0,NEUVIS PRO SYRU,0,False,1
7,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR,4.215603,2025-07-30,False,5.0,3.0,5.0,3.174667,8.0,8.174667,1,VOMETA DROPS 5,1,False,2
8,A-19155-00,VOMETA DROPS 5 MG/ML 10 ML MYANMAR (2),4.215603,2025-07-30,False,3.0,3.0,1.0,3.174667,6.0,4.174667,2,VOMETA DROPS 5,1,False,2
9,A-14593-00,RHINOS NEO DROPS 10 ML,4.595897,2025-07-30,False,5.0,3.0,5.0,19.047667,8.0,24.047667,45,RHINOS NEO DROP,9,False,3


In [180]:
dfUtil = dfSorted2.copy()
# dfUtil.to_csv(f"{month} PPIC Runtimes.csv",index = False)
# files.download(f"{month} PPIC Runtimes.csv")

## Gemini

### Trial 1

In [181]:

# def schedule_liquid_line(df_batches, month_year, overtime=False):

#     # --- Configuration ---
#     # Define working hours based on overtime flag
#     if overtime:
#         # 24/7 operation
#         DAILY_WORKING_HOURS = 24.0
#         DAILY_START_TIME = timedelta(hours=0, minutes=0)
#         DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
#     else:
#         # Shift 1 (7:15 AM - 4:00 PM) + Shift 2 (3:30 PM - 11 :45 PM) combined
#         # Continuous operation from 7:15 AM to 11:45 PM
#         DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#         DAILY_START_TIME = timedelta(hours=7, minutes=15)
#         DAILY_END_TIME = timedelta(hours=23, minutes=45)

#     # Convert Due Date to datetime objects
#     df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

#     # Initialize scheduling variables
#     # Start of the scheduling month
#     start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

#     # Initialize machine availability times
#     # Both machines are available at the start of the first working day
#     current_time = start_of_month + DAILY_START_TIME

#     # Ensure current_time is on a weekday if not overtime
#     if not overtime and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#         current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday

#     mixing_machine_available_at = current_time
#     filling_machine_available_at = current_time

#     # List to store all events for the final DataFrame
#     events = []

#     # Flag for the first batch of the month
#     is_first_batch = True

#     # --- Helper Function to find next available start time for an operation ---
#     def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date):
#         """
#         Calculates the earliest time an operation can start, respecting shifts and continuous run.
#         """
#         proposed_start_time = current_machine_time

#         while True:
#             # Check if proposed_start_time is within the current day's working window
#             day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                     second=0, microsecond=0)
#             day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
#                                                   minute=(DAILY_END_TIME.seconds % 3600) // 60,
#                                                   second=0, microsecond=0)

#             # If not overtime, skip weekends
#             if not overtime and proposed_start_time.weekday() >= 5: # Saturday or Sunday
#                 proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                 proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                   minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                   second=0, microsecond=0)
#                 continue # Re-check the new proposed_start_time

#             # Ensure proposed_start_time is not before the daily start time
#             if proposed_start_time < day_start:
#                 proposed_start_time = day_start

#             # Calculate potential end time
#             potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

#             # Check if the operation fits within the current day's working window
#             if potential_end_time <= day_end + timedelta(minutes=1): # Add a minute buffer for exact end time
#                 # Check against due date
#                 if potential_end_time.date() > current_batch_due_date.date():
#                     print(f"Warning: Batch cannot be completed by due date {current_batch_due_date.date()}.")
#                     return None # Indicate impossible to schedule within due date
#                 return proposed_start_time
#             else:
#                 # Operation spills over, move to the start of the next working day
#                 proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
#                     hour=DAILY_START_TIME.seconds // 3600,
#                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                     second=0, microsecond=0
#                 )
#                 # Ensure it's not a weekend if not overtime
#                 if not overtime and proposed_start_time.weekday() >= 5:
#                     proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                     proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                       minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                       second=0, microsecond=0)

#     # --- Schedule Each Batch ---
#     for index, row in df_batches.iterrows():
#         item_number = row['Item Number']
#         batch_name = row['Batch']
#         mixing_runtime = row['Total Mixing Runtime']
#         filling_runtime = row['Total Filling Runtime']
#         due_date = row['Due Date']

#         # --- Schedule Mixing ---
#         start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date)
#         if start_mixing_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to due date constraint for mixing.")
#             continue # Cannot schedule this batch
#         end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
#         mixing_machine_available_at = end_mixing_time # Update mixing machine availability

#         # --- Schedule Filling ---
#         # Filling machine must be available AND (if first batch, mixing must be done)
#         filling_start_constraint = max(filling_machine_available_at, start_mixing_time if is_first_batch else filling_machine_available_at)

#         start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date)
#         if start_filling_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to due date constraint for filling.")
#             continue # Cannot schedule this batch
#         end_filling_time = start_filling_time + timedelta(hours=filling_runtime)
#         filling_machine_available_at = end_filling_time # Update filling machine availability

#         # Update first batch flag
#         if is_first_batch:
#             is_first_batch = False

#         # --- Record Events ---
#         events.append({
#             'Timestamp': start_mixing_time,
#             'Start Mixing Item Number': item_number,
#             'Start Mixing Batch': batch_name,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_mixing_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': item_number,
#             'End Mixing Batch': batch_name,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': start_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': item_number,
#             'Start Filling Batch': batch_name,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': item_number,
#             'End Filling Batch': batch_name,
#             'End Filling Batch Due Date': due_date
#         })

#     # Create the final DataFrame
#     df_schedule = pd.DataFrame(events)
#     df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

#     # Fill NaNs where no event occurs at a specific timestamp (as per your request)
#     # This step is implicitly handled by how we construct `events`
#     # However, if multiple events happen at the exact same timestamp, they will be separate rows.
#     # If you want to consolidate them into one row, that would require a different aggregation.
#     # For now, each event (start/end) gets its own row.

#     return df_schedule

# # --- Example Usage ---
# # Your dataset (transcribed from the image)
# df_data = pd.DataFrame({
#     'Item Number': [1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008],
#     'Batch': ['Product A', 'Product B', 'Product C', 'Product D', 'Product E', 'Product F', 'Product G', 'Product H'],
#     'Total Mixing Runtime (hours)': [5, 6, 4, 7, 5, 6, 4, 7],
#     'Total Filling Runtime (hours)': [4, 5, 3, 6, 4, 5, 3, 6],
#     'Due Date (YYYY-MM-DD)': ['2024-07-05', '2024-07-08', '2024-07-12', '2024-07-15',
#                               '2024-07-18', '2024-07-22', '2024-07-25', '2024-07-29']
# })


# # --- Run the scheduler for July 2024 ---


# df_schedule_standard = schedule_liquid_line(dfSched, '2025-07', overtime)
# #print(df_schedule_standard.to_string())
# data_table.DataTable(df_schedule_standard)

### Trial 2

#### Functions

In [182]:
# def schedule_liquid_line(df_batches, month_year, overtime=False, feasible = True, shiftNo = '2 Shifts'):

#     # Define working hours based on overtime flag
#     if shiftNo == '3 Shifts':
#         # 24/7 operation
#         DAILY_WORKING_HOURS = 24.0
#         DAILY_START_TIME = timedelta(hours=0, minutes=0)
#         DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
#     else:
#         # Shift 1 (7:15 AM - 4:00 PM) + Shift 2 (3:30 PM - 11:45 PM) combined
#         # Continuous operation from 7:15 AM to 11:45 PM
#         DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#         DAILY_START_TIME = timedelta(hours=7, minutes=15)
#         DAILY_END_TIME = timedelta(hours=23, minutes=45)

#     # #Shift 1 (7:15 AM - 4:00 PM) + Shift 2 (3:30 PM - 11:45 PM) combined
#     # # Continuous operation from 7:15 AM to 11:45 PM
#     # DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#     # DAILY_START_TIME = timedelta(hours=7, minutes=15)
#     # DAILY_END_TIME = timedelta(hours=23, minutes=45)

#     # PPIC Feasibility for current month
#     if not feasible:
#       print("Optimally, PPIC MS cannot be met, feasibility: False\n")

#     # Convert Due Date to datetime objects
#     df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

#     # Initialize scheduling variables
#     # Start of the scheduling month
#     start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

#     # Initialize machine availability times
#     # Both machines are available at the start of the first working day
#     current_time = start_of_month + DAILY_START_TIME

#     # Ensure current_time is on a weekday if not overtime
#     if not overtime and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#         current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday

#     mixing_machine_available_at = current_time
#     filling_machine_available_at = current_time

#     # List to store all events for the final DataFrame
#     events = []

#     # --- Helper Function to find next available start time for an operation ---
#     # Added initial_scheduling_start_time to the signature for the safeguard
#     def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date, initial_scheduling_start_time, item_number, batch_name, operation_type):
#         """
#         Calculates the earliest time an operation can start, respecting shifts and continuous run.
#         """
#         proposed_start_time = current_machine_time

#         # Safeguard: Prevent infinite loop or date overflow
#         # If proposed start time is too far beyond the initial scheduling start, something is wrong.
#         # Max search duration set to 5 years as a safety net.
#         max_search_days = 365 * 5

#         while True:
#             # Check for excessive search duration before potential overflow
#             if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
#                 print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} exceeded max search duration ({max_search_days} days). Aborting.")
#                 return None # Indicate that a slot couldn't be found within a reasonable timeframe

#             # Check if proposed_start_time is within the current day's working window
#             day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                     second=0, microsecond=0)
#             day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
#                                                   minute=(DAILY_END_TIME.seconds % 3600) // 60,
#                                                   second=0, microsecond=0)

#             # If not overtime, skip weekends
#             if not overtime and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#                 proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                 proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                   minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                   second=0, microsecond=0)
#                 continue # Re-check the new proposed_start_time

#             # Ensure proposed_start_time is not before the daily start time
#             if proposed_start_time < day_start:
#                 proposed_start_time = day_start

#             # # DEBUGGING PRINTS:
#             # print(f"DEBUG: Batch {item_number} - {batch_name} ({operation_type})")
#             # print(f"DEBUG:   proposed_start_time = {proposed_start_time}")
#             # print(f"DEBUG:   duration_hours = {duration_hours}")

#             # Calculate potential end time
#             potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

#             # Check if the operation fits within the current day's working window
#             if potential_end_time <= day_end + timedelta(minutes=1): # Add a minute buffer for exact end time
#                 # Check against due date
#                 if potential_end_time.date() > current_batch_due_date.date():
#                     print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
#                     return None # Indicate impossible to schedule within due date
#                 return proposed_start_time
#             else:
#                 # Operation spills over, move to the start of the next working day
#                 proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
#                     hour=DAILY_START_TIME.seconds // 3600,
#                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                     second=0, microsecond=0
#                 )
#                 # Ensure it's not a weekend if not overtime
#                 if not overtime and proposed_start_time.weekday() >= 5:
#                     proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                     proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                       minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                       second=0, microsecond=0)
#     # Flag for the first batch of the month
#     is_first_batch = True

#     # --- Schedule Each Batch ---
#     for index, row in df_batches.iterrows():
#         item_number = row['Item Number']
#         batch_name = row['Batch']
#         mixing_runtime = row['Total Mixing Runtime']
#         filling_runtime = row['Total Filling Runtime']
#         due_date = row['Due Date']

#         # --- Schedule Mixing ---
#         start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date, start_of_month, item_number, batch_name, "Mixing")
#         if start_mixing_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues.")
#             continue # Cannot schedule this batch
#         end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
#         mixing_machine_available_at = end_mixing_time # Update mixing machine availability

#         # --- Schedule Filling ---
#         # CORRECTED LOGIC FOR FIRST BATCH DEPENDENCY
#         if is_first_batch:
#             # For the first batch, filling cannot start until its mixing is complete.
#             filling_start_constraint = max(filling_machine_available_at, end_mixing_time)
#         else:
#             # For other batches, filling can start when the filling machine is available
#             # (mixing for THIS batch might be happening concurrently with filling of a PREVIOUS batch)
#             filling_start_constraint = filling_machine_available_at


#         start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date, start_of_month, item_number, batch_name, "Filling")
#         if start_filling_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues.")
#             continue # Cannot schedule this batch
#         end_filling_time = start_filling_time + timedelta(hours=filling_runtime)
#         filling_machine_available_at = end_filling_time # Update filling machine availability

#         # Update first batch flag
#         if is_first_batch:
#             is_first_batch = False

#         # --- Record Events ---
#         events.append({
#             'Timestamp': start_mixing_time,
#             'Start Mixing Item Number': item_number,
#             'Start Mixing Batch': batch_name,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_mixing_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': item_number,
#             'End Mixing Batch': batch_name,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': start_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': item_number,
#             'Start Filling Batch': batch_name,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': item_number,
#             'End Filling Batch': batch_name,
#             'End Filling Batch Due Date': due_date
#         })

#     # Create the final DataFrame
#     df_schedule = pd.DataFrame(events)
#     df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

#     # Fill NaNs where no event occurs at a specific timestamp (as per your request)
#     # This step is implicitly handled by how we construct `events`
#     # However, if multiple events happen at the exact same timestamp, they will be separate rows.
#     # If you want to consolidate them into one row, that would require a different aggregation.
#     # For now, each event (start/end) gets its own row.

#     return df_schedule


#### Run Schedule

In [183]:
# # --- Run the scheduler for July 2024 ---
# dfSched = dfUtil.copy()

# overtime = True

# print(f"\n--- Scheduling with Overtime {overtime} ---")
# df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, shiftNo)


# data_table.DataTable(df_schedule_overtime)

#### Debugging

In [184]:
# import pandas as pd
# from datetime import datetime, timedelta
# import numpy as np

# def schedule_liquid_line(df_batches, month_year, overtime=False):
#     """
#     Schedules batches for a liquid production line based on given constraints.

#     Args:
#         df_batches (pd.DataFrame): DataFrame with batch details including:
#             - 'Item Number': Batch's code
#             - 'Batch': Name of the product
#             - 'Total Mixing Runtime (hours)': Time required for mixing
#             - 'Total Filling Runtime (hours)': Time required for filling
#             - 'Due Date (YYYY-MM-DD)': Due date for the batch
#         month_year (str): The month and year for scheduling (e.g., '2024-07').
#         overtime (bool): If True, schedule 24/7. If False, schedule 7:15 AM - 11:45 PM on weekdays.

#     Returns:
#         pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
#     """

#     # --- Configuration ---
#     # Define working hours based on overtime flag
#     if overtime:
#         # 24/7 operation
#         DAILY_WORKING_HOURS = 24.0
#         DAILY_START_TIME = timedelta(hours=0, minutes=0)
#         DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
#     else:
#         # Shift 1 (7:15 AM - 4:00 PM) + Shift 2 (3:30 PM - 11:45 PM) combined
#         # Continuous operation from 7:15 AM to 11:45 PM
#         DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#         DAILY_START_TIME = timedelta(hours=7, minutes=15)
#         DAILY_END_TIME = timedelta(hours=23, minutes=45)

#     # Convert Due Date to datetime objects
#     df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

#     # Initialize scheduling variables
#     # Start of the scheduling month
#     start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

#     # Initialize machine availability times
#     # Both machines are available at the start of the first working day
#     current_time = start_of_month + DAILY_START_TIME

#     # Ensure current_time is on a weekday if not overtime
#     if not overtime and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#         current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday

#     mixing_machine_available_at = current_time
#     filling_machine_available_at = current_time

#     # List to store all events for the final DataFrame
#     events = []

#     # Flag for the first batch of the month
#     is_first_batch = True

#     # --- Helper Function to find next available start time for an operation ---
#     # Added initial_scheduling_start_time to the signature for the safeguard
#     def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date, initial_scheduling_start_time, item_number, batch_name, operation_type):
#         """
#         Calculates the earliest time an operation can start, respecting shifts and continuous run.
#         """
#         proposed_start_time = current_machine_time

#         # Safeguard: Prevent infinite loop or date overflow
#         # If proposed start time is too far beyond the initial scheduling start, something is wrong.
#         # Max search duration set to 5 years as a safety net.
#         max_search_days = 365 * 5

#         while True:
#             # Check for excessive search duration before potential overflow
#             if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
#                 print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} exceeded max search duration ({max_search_days} days). Aborting.")
#                 return None # Indicate that a slot couldn't be found within a reasonable timeframe

#             # Check if proposed_start_time is within the current day's working window
#             day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                     second=0, microsecond=0)
#             day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
#                                                   minute=(DAILY_END_TIME.seconds % 3600) // 60,
#                                                   second=0, microsecond=0)

#             # If not overtime, skip weekends
#             if not overtime and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#                 proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                 proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                   minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                   second=0, microsecond=0)
#                 continue # Re-check the new proposed_start_time

#             # Ensure proposed_start_time is not before the daily start time
#             if proposed_start_time < day_start:
#                 proposed_start_time = day_start

#             # # DEBUGGING PRINTS:
#             # print(f"DEBUG: Batch {item_number} - {batch_name} ({operation_type})")
#             # print(f"DEBUG:   proposed_start_time = {proposed_start_time}")
#             # print(f"DEBUG:   duration_hours = {duration_hours}")

#             # Calculate potential end time
#             potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

#             # Check if the operation fits within the current day's working window
#             if potential_end_time <= day_end + timedelta(minutes=1): # Add a minute buffer for exact end time
#                 # Check against due date
#                 if potential_end_time.date() > current_batch_due_date.date():
#                     print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
#                     return None # Indicate impossible to schedule within due date
#                 return proposed_start_time
#             else:
#                 # Operation spills over, move to the start of the next working day
#                 proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
#                     hour=DAILY_START_TIME.seconds // 3600,
#                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                     second=0, microsecond=0
#                 )
#                 # Ensure it's not a weekend if not overtime
#                 if not overtime and proposed_start_time.weekday() >= 5:
#                     proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                     proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                       minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                       second=0, microsecond=0)

#     # --- Schedule Each Batch ---
#     for index, row in df_batches.iterrows():
#         item_number = row['Item Number']
#         batch_name = row['Batch']
#         mixing_runtime = row['Total Mixing Runtime']
#         filling_runtime = row['Total Filling Runtime']
#         due_date = row['Due Date']

#         # --- Schedule Mixing ---
#         start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date, start_of_month, item_number, batch_name, "Mixing")
#         if start_mixing_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues.")
#             continue # Cannot schedule this batch
#         end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
#         mixing_machine_available_at = end_mixing_time # Update mixing machine availability

#         # --- Schedule Filling ---
#         # Filling machine must be available AND (if first batch, mixing must be done)
#         filling_start_constraint = max(filling_machine_available_at, start_mixing_time if is_first_batch else filling_machine_available_at)

#         start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date, start_of_month, item_number, batch_name, "Filling")
#         if start_filling_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues.")
#             continue # Cannot schedule this batch
#         end_filling_time = start_filling_time + timedelta(hours=filling_runtime)
#         filling_machine_available_at = end_filling_time # Update filling machine availability

#         # Update first batch flag
#         if is_first_batch:
#             is_first_batch = False

#         # --- Record Events ---
#         events.append({
#             'Timestamp': start_mixing_time,
#             'Start Mixing Item Number': item_number,
#             'Start Mixing Batch': batch_name,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_mixing_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': item_number,
#             'End Mixing Batch': batch_name,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': start_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': item_number,
#             'Start Filling Batch': batch_name,
#             'End Filling Item Number': np.nan,
#             'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         })
#         events.append({
#             'Timestamp': end_filling_time,
#             'Start Mixing Item Number': np.nan,
#             'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan,
#             'End Mixing Batch': np.nan,
#             'Start Filling Item Number': np.nan,
#             'Start Filling Batch': np.nan,
#             'End Filling Item Number': item_number,
#             'End Filling Batch': batch_name,
#             'End Filling Batch Due Date': due_date
#         })

#     # Create the final DataFrame
#     df_schedule = pd.DataFrame(events)
#     df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

#     # Fill NaNs where no event occurs at a specific timestamp (as per your request)
#     # This step is implicitly handled by how we construct `events`
#     # However, if multiple events happen at the exact same timestamp, they will be separate rows.
#     # If you want to consolidate them into one row, that would require a different aggregation.
#     # For now, each event (start/end) gets its own row.

#     return df_schedule

# # --- Example Usage ---
# # Your dataset (transcribed from the image)
# df_data = pd.DataFrame({
#     'Item Number': [1001, 1003, 1004, 1005, 1006, 1007, 1008, 1002], # Changed order of 1002
#     'Batch': ['Product A', 'Product C', 'Product D', 'Product E', 'Product F', 'Product G', 'Product H', 'Product B'], # Changed order of Product B
#     'Total Mixing Runtime (hours)': [5, 4, 7, 5, 6, 4, 7, 6],
#     'Total Filling Runtime (hours)': [4, 3, 6, 4, 5, 3, 6, 5],
#     'Due Date (YYYY-MM-DD)': ['2024-07-05', '2024-07-12', '2024-07-15', '2024-07-18',
#                               '2024-07-22', '2024-07-25', '2024-07-29', '2024-07-08'] # Changed order of 2024-07-08
# })

# # --- Run the scheduler for July 2024 ---

# # print("--- Scheduling with Standard Shifts (Weekdays, 7:15 AM - 11:45 PM) ---")
# # # Assuming dfSched is defined in your environment, otherwise use df_data
# # df_schedule_standard = schedule_liquid_line(df_data.copy(), '2024-07', overtime=False)
# # print(df_schedule_standard.to_string())

# # print("\n--- Scheduling with Overtime (24/7) ---")

# df_schedule_overtime = schedule_liquid_line(dfSched, '2025-07', overtime)
# #print(df_schedule_overtime.to_string())
# data_table.DataTable(df_schedule_overtime)

### Trial 3

#### Functions

In [185]:

def schedule_liquid_line(df_batches, month_year, overtime=False, feasible=True, shiftNo='2 Shifts'):
    """
    Schedules batches for a liquid production line based on given constraints.

    Args:a
        df_batches (pd.DataFrame): DataFrame with batch details including:
            - 'Item Number': Batch's code
            - 'Batch': Name of the product
            - 'Total Mixing Runtime': Time required for mixing
            - 'Total Filling Runtime': Time required for filling
            - 'Due Date': Due date for the batch
            - 'IsBulk' (optional, bool): If True, indicates a bulk item not requiring mixing columns.
        month_year (str): The month and year for scheduling (e.g., '2024-07').
        overtime (bool): If True, allows scheduling on weekends (only applicable if shiftNo is not '3 Shifts').
                         If shiftNo is '3 Shifts', overtime is implicitly True for weekends.
        feasible (bool): If False, prints a message indicating PPIC MS cannot be met.
        shiftNo (str): Defines the daily working shifts:
                       '3 Shifts': 24/7 operation.
                       '2 Shifts': 7:15 AM - 11:45 PM.

    Returns:
        pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
    """

    # --- Configuration ---
    # Define working hours based on shiftNo
    if shiftNo == '3 Shifts':
        DAILY_WORKING_HOURS = 24.0
        DAILY_START_TIME = timedelta(hours=0, minutes=0)
        DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
        allow_weekend_work_in_helper = True # 3 Shifts implies 24/7, including weekends
    else: # Default to '2 Shifts' (or any other value)
        DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
        DAILY_START_TIME = timedelta(hours=7, minutes=15)
        DAILY_END_TIME = timedelta(hours=23, minutes=45)
        allow_weekend_work_in_helper = overtime # Weekends worked only if 'overtime' flag is True

    # PPIC Feasibility for current month
    if not feasible:
        print("Optimally, PPIC MS cannot be met, feasibility: False\n")

    # Convert Due Date to datetime objects
    df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

    # Initialize scheduling variables
    # Start of the scheduling month
    start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

    # Initialize machine availability times
    current_time = start_of_month + DAILY_START_TIME

    # Ensure current_time is on a weekday if not allowing weekend work initially
    if not allow_weekend_work_in_helper and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
        current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday
        current_time = current_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                            minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                            second=0, microsecond=0)


    mixing_machine_available_at = current_time
    filling_machine_available_at = current_time

    # List to store all events for the final DataFrame
    events = []

    # BUG FIX: is_first_batch must be initialized OUTSIDE the loop
    is_first_batch = True

    # --- Helper Function to find next available start time for an operation ---
    def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date,
                                initial_scheduling_start_time, item_number, batch_name,
                                operation_type, allow_weekend_work):
        """
        Calculates the earliest time an operation can start, respecting shifts and continuous run.
        """
        proposed_start_time = current_machine_time

        # Safeguard: Prevent infinite loop or date overflow
        max_search_days = 365 * 5

        while True:
            if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
                print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} exceeded max search duration ({max_search_days} days). Aborting.")
                return None

            day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                    second=0, microsecond=0)
            day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
                                                  minute=(DAILY_END_TIME.seconds % 3600) // 60,
                                                  second=0, microsecond=0)

            # If not allowing weekend work, skip weekends
            if not allow_weekend_work and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
                proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                  minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                  second=0, microsecond=0)
                continue # Re-check the new proposed_start_time

            # Ensure proposed_start_time is not before the daily start time
            if proposed_start_time < day_start:
                proposed_start_time = day_start

            potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

            if potential_end_time <= day_end + timedelta(minutes=1):
                if potential_end_time.date() > current_batch_due_date.date():
                    print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
                    return None
                return proposed_start_time
            else:
                # Operation spills over, move to the start of the next working day
                proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
                    hour=DAILY_START_TIME.seconds // 3600,
                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                    second=0, microsecond=0
                )
                # Ensure it's not a weekend if not allowing weekend work
                if not allow_weekend_work and proposed_start_time.weekday() >= 5:
                    proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                    proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                      minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                      second=0, microsecond=0)

    # --- Schedule Each Batch ---
    for index, row in df_batches.iterrows():
        item_number = row['Item Number']
        batch_name = row['Batch']
        mixing_runtime = row['Total Mixing Runtime']
        filling_runtime = row['Total Filling Runtime']
        due_date = row['Due Date']

        # Check for bulk status for mixing
        is_bulk_for_mixing = row.get('Bulk', False) # Defaults to False if 'IsBulk' column doesn't exist

        # Check for 'bulk' in batch name for filling (case-insensitive)
        is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()


        # --- Schedule Mixing ---
        # Only schedule mixing if it's not a bulk item (based on 'IsBulk' column)
        if not is_bulk_for_mixing:
            start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
                                                        start_of_month, item_number, batch_name, "Mixing",
                                                        allow_weekend_work_in_helper)
            if start_mixing_time is None:
                print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
                continue
            end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
            mixing_machine_available_at = end_mixing_time
        else:
            # If it's a bulk item for mixing, assume mixing is instantaneous or not applicable
            # Set times to current machine availability to not block other operations
            start_mixing_time = mixing_machine_available_at
            end_mixing_time = mixing_machine_available_at
            #print(f"Batch {item_number} - {batch_name} marked as bulk, skipping mixing scheduling.")


        # --- Schedule Filling ---
        # CORRECTED LOGIC FOR FIRST BATCH DEPENDENCY
        if is_first_batch:
            # For the first batch, filling cannot start until its mixing is complete.
            # This applies even if mixing was 'skipped' for bulk, ensuring sequential flow for the first.
            filling_start_constraint = max(filling_machine_available_at, end_mixing_time)
        else:
            # For other batches, filling can start when the filling machine is available
            # (mixing for THIS batch might be happening concurrently with filling of a PREVIOUS batch)
            filling_start_constraint = filling_machine_available_at

        # Only schedule filling if 'bulk' is not in the batch name
        if not is_bulk_in_name_for_filling:
            start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date,
                                                         start_of_month, item_number, batch_name, "Filling",
                                                         allow_weekend_work_in_helper)
            if start_filling_time is None:
                print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Filling.")
                continue
            end_filling_time = start_filling_time + timedelta(hours=filling_runtime)
            filling_machine_available_at = end_filling_time
        else:
            # If 'bulk' in name, assume filling is instantaneous or not applicable
            # Set times to current machine availability to not block other operations
            start_filling_time = filling_machine_available_at
            end_filling_time = filling_machine_available_at
            #print(f"Batch {item_number} - {batch_name} contains 'bulk' in name, skipping filling scheduling.")


        # Update first batch flag
        if is_first_batch:
            is_first_batch = False

        # --- Record Events ---
        # Record mixing events only if not a bulk item for mixing
        if not is_bulk_for_mixing:
            events.append({
                'Timestamp': start_mixing_time,
                'Start Mixing Item Number': item_number,
                'Start Mixing Batch': batch_name,
                'End Mixing Item Number': np.nan,
                'End Mixing Batch': np.nan,
                'Start Filling Item Number': np.nan,
                'Start Filling Batch': np.nan,
                'End Filling Item Number': np.nan,
                'End Filling Batch': np.nan,
                'End Filling Batch Due Date': due_date
            })
            events.append({
                'Timestamp': end_mixing_time,
                'Start Mixing Item Number': np.nan,
                'Start Mixing Batch': np.nan,
                'End Mixing Item Number': item_number,
                'End Mixing Batch': batch_name,
                'Start Filling Item Number': np.nan,
                'Start Filling Batch': np.nan,
                'End Filling Item Number': np.nan,
                'End Filling Batch': np.nan,
                'End Filling Batch Due Date': due_date
            })

        # Record filling events only if 'bulk' is not in the batch name
        if not is_bulk_in_name_for_filling:
            events.append({
                'Timestamp': start_filling_time,
                'Start Mixing Item Number': np.nan,
                'Start Mixing Batch': np.nan,
                'End Mixing Item Number': np.nan,
                'End Mixing Batch': np.nan,
                'Start Filling Item Number': item_number,
                'Start Filling Batch': batch_name,
                'End Filling Item Number': np.nan,
                'End Filling Batch': np.nan,
                'End Filling Batch Due Date': due_date
            })
            events.append({
                'Timestamp': end_filling_time,
                'Start Mixing Item Number': np.nan,
                'Start Mixing Batch': np.nan,
                'End Mixing Item Number': np.nan,
                'End Mixing Batch': np.nan,
                'Start Filling Item Number': np.nan,
                'Start Filling Batch': np.nan,
                'End Filling Item Number': item_number,
                'End Filling Batch': batch_name,
                'End Filling Batch Due Date': due_date
            })

    # Create the final DataFrame
    df_schedule = pd.DataFrame(events)
    df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

    return df_schedule


#### Testing

In [186]:
# --- Run the scheduler for July 2025 ---
dfSched = dfUtil.copy()

overtime = True

print(f"\n--- Scheduling with Overtime {overtime} ---")
df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, "2 Shifts")


data_table.DataTable(df_schedule_overtime)



--- Scheduling with Overtime True ---
Error: Scheduling for Filling of batch A-14593-00 - RHINOS NEO DROPS 10 ML exceeded max search duration (1825 days). Aborting.
Skipping batch A-14593-00 - RHINOS NEO DROPS 10 ML due to scheduling issues for Filling.
Error: Scheduling for Filling of batch A-14593-00 - RHINOS NEO DROPS 10 ML (2) exceeded max search duration (1825 days). Aborting.
Skipping batch A-14593-00 - RHINOS NEO DROPS 10 ML (2) due to scheduling issues for Filling.
Error: Scheduling for Filling of batch A-14594-00 - RHINOS JUNIOR SYRUP 60 ML exceeded max search duration (1825 days). Aborting.
Skipping batch A-14594-00 - RHINOS JUNIOR SYRUP 60 ML due to scheduling issues for Filling.
Error: Scheduling for Filling of batch A-14594-00 - RHINOS JUNIOR SYRUP 60 ML (4) exceeded max search duration (1825 days). Aborting.
Skipping batch A-14594-00 - RHINOS JUNIOR SYRUP 60 ML (4) due to scheduling issues for Filling.
Error: Scheduling for Filling of batch A-17029-00 - LYTACUR SUSPENSIO

,Timestamp,Start Mixing Item Number,Start Mixing Batch,End Mixing Item Number,End Mixing Batch,Start Filling Item Number,Start Filling Batch,End Filling Item Number,End Filling Batch,End Filling Batch Due Date
0,2025-07-01 07:15:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
1,2025-07-01 09:15:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,2025-07-30
2,2025-07-01 09:15:00.000000,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,2025-07-30
3,2025-07-01 09:15:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
4,2025-07-01 12:15:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),NaN,NaN,NaN,NaN,2025-07-30
5,2025-07-01 12:15:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (3),NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
6,2025-07-01 15:15:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (3),NaN,NaN,NaN,NaN,2025-07-30
7,2025-07-01 15:15:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (4),NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
8,2025-07-01 18:15:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (5),NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
9,2025-07-01 18:15:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (4),NaN,NaN,NaN,NaN,2025-07-30


### WIP Integration

#### Function

In [187]:

# def schedule_liquid_line(df_batches, month_year, overtime=False, feasible=True, shiftNo='2 Shifts'):
#     """
#     Schedules batches for a liquid production line based on given constraints.

#     Args:
#         df_batches (pd.DataFrame): DataFrame with batch details including:
#             - 'Item Number': Batch's code
#             - 'Batch': Name of the product
#             - 'Total Mixing Runtime': Time required for mixing
#             - 'Total Filling Runtime': Time required for filling
#             - 'Due Date': Due date for the batch
#             - 'IsBulk' (optional, bool): If True, indicates a bulk item not requiring mixing columns.
#         month_year (str): The month and year for scheduling (e.g., '2024-07').
#         overtime (bool): If True, allows scheduling on weekends (only applicable if shiftNo is not '3 Shifts').
#                          If shiftNo is '3 Shifts', overtime is implicitly True for weekends.
#         feasible (bool): If False, prints a message indicating PPIC MS cannot be met.
#         shiftNo (str): Defines the daily working shifts:
#                        '3 Shifts': 24/7 operation.
#                        '2 Shifts': 7:15 AM - 11:45 PM.

#     Returns:
#         pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
#     """

#     # --- Configuration ---
#     # Define working hours based on shiftNo
#     if shiftNo == '3 Shifts':
#         DAILY_WORKING_HOURS = 24.0
#         DAILY_START_TIME = timedelta(hours=0, minutes=0)
#         DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
#         allow_weekend_work_in_helper = True # 3 Shifts implies 24/7, including weekends
#     else: # Default to '2 Shifts' (or any other value)
#         DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#         DAILY_START_TIME = timedelta(hours=7, minutes=15)
#         DAILY_END_TIME = timedelta(hours=23, minutes=45)
#         allow_weekend_work_in_helper = overtime # Weekends worked only if 'overtime' flag is True

#     # PPIC Feasibility for current month
#     if not feasible:
#         print("Optimally, PPIC MS cannot be met, feasibility: False\n")

#     # Convert Due Date to datetime objects
#     df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

#     # Initialize scheduling variables
#     # Start of the scheduling month
#     start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

#     # Initialize machine availability times
#     current_time = start_of_month + DAILY_START_TIME

#     # Ensure current_time is on a weekday if not allowing weekend work initially
#     if not allow_weekend_work_in_helper and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#         current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday
#         current_time = current_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                             minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                             second=0, microsecond=0)

#     mixing_machine_available_at = current_time
#     filling_machine_available_at = current_time

#     # Initialize holding tank availability (list of timestamps when each tank is free)
#     # Each element represents when that specific tank will be free.
#     holding_tank_free_at = [current_time, current_time, current_time] # 3 tanks

#     # List to store all events for the final DataFrame
#     events = []

#     # BUG FIX: is_first_batch must be initialized OUTSIDE the loop
#     is_first_batch = True

#     # --- Helper Function to find next available start time for an operation ---
#     def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date,
#                                 initial_scheduling_start_time, item_number, batch_name,
#                                 operation_type, allow_weekend_work):
#         """
#         Calculates the earliest time an operation can start, respecting shifts and continuous run.
#         """
#         proposed_start_time = current_machine_time

#         # Safeguard: Prevent infinite loop or date overflow
#         max_search_days = 365 * 5

#         while True:
#             if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
#                 print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} exceeded max search duration ({max_search_days} days). Aborting.")
#                 return None

#             day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                     second=0, microsecond=0)
#             day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
#                                                   minute=(DAILY_END_TIME.seconds % 3600) // 60,
#                                                   second=0, microsecond=0)

#             # If not allowing weekend work, skip weekends
#             if not allow_weekend_work and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#                 proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                 proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                   minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                   second=0, microsecond=0)
#                 continue # Re-check the new proposed_start_time

#             # Ensure proposed_start_time is not before the daily start time
#             if proposed_start_time < day_start:
#                 proposed_start_time = day_start

#             potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

#             if potential_end_time <= day_end + timedelta(minutes=1):
#                 if potential_end_time.date() > current_batch_due_date.date():
#                     print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
#                     return None
#                 return proposed_start_time
#             else:
#                 # Operation spills over, move to the start of the next working day
#                 proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
#                     hour=DAILY_START_TIME.seconds // 3600,
#                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                     second=0, microsecond=0
#                 )
#                 # Ensure it's not a weekend if not allowing weekend work
#                 if not allow_weekend_work and proposed_start_time.weekday() >= 5:
#                     proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                     proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                       minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                       second=0, microsecond=0)

#     # --- Schedule Each Batch ---
#     for index, row in df_batches.iterrows():
#         item_number = row['Item Number']
#         batch_name = row['Batch']
#         mixing_runtime = row['Total Mixing Runtime']
#         filling_runtime = row['Total Filling Runtime']
#         due_date = row['Due Date']

#         # Check for bulk status for mixing
#         is_bulk_for_mixing = row.get('Bulk', False) # Defaults to False if 'Bulk' column doesn't exist or is not True

#         # Check for 'bulk' in batch name for filling (case-insensitive)
#         is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()


#         # --- Schedule Mixing ---
#         # Only schedule mixing if it's not a bulk item (based on 'Bulk' column)
#         if not is_bulk_for_mixing:
#             start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
#                                                         start_of_month, item_number, batch_name, "Mixing",
#                                                         allow_weekend_work_in_helper)
#             if start_mixing_time is None:
#                 print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
#                 continue
#             end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
#             mixing_machine_available_at = end_mixing_time
#         else:
#             # If it's a bulk item for mixing, assume mixing is instantaneous or not applicable
#             # Set times to current machine availability to not block other operations
#             start_mixing_time = mixing_machine_available_at
#             end_mixing_time = mixing_machine_available_at
#             #print(f"Batch {item_number} - {batch_name} marked as bulk, skipping mixing scheduling.")


#         # --- Schedule Quality Check ---
#         # QC starts immediately after mixing ends (or "virtual" end_mixing_time for bulk)
#         # QC takes 8 hours for all batches
#         start_qc_time = end_mixing_time
#         end_qc_time = start_qc_time + timedelta(hours=8)

#         # Check if QC completion is past due date
#         if end_qc_time.date() > due_date.date():
#             print(f"Warning: Batch {item_number} - {batch_name} QC cannot be completed by due date {due_date.date()}. Skipping.")
#             continue


#         # --- Schedule Holding Tank Entry ---
#         # Find the earliest available holding tank
#         earliest_free_tank_time = min(holding_tank_free_at)
#         tank_index = holding_tank_free_at.index(earliest_free_tank_time) # Get the index of the free tank

#         # Batch enters holding tank after QC AND after the tank is free
#         tank_entry_time = max(end_qc_time, earliest_free_tank_time)

#         # --- Schedule Filling ---
#         # The earliest this batch can start filling is:
#         # 1. When the filling machine is free (filling_machine_available_at)
#         # 2. When the batch has finished QC and entered a tank (tank_entry_time)
#         # 3. For the first batch, it must also wait for its own mixing/QC (handled by end_qc_time which is based on end_mixing_time)

#         # The constraint for filling start from the batch's perspective (after QC and tank entry)
#         batch_ready_for_filling_time = tank_entry_time

#         # If it's the first batch, ensure it waits for its own mixing/QC completion
#         if is_first_batch:
#             filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)
#         else:
#             # For subsequent batches, filling machine can work on this batch
#             # as soon as it's ready AND filling machine is free.
#             filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)

#         # Check the 7-day max holding time constraint for filling start
#         max_allowed_filling_start_from_qc = end_qc_time + timedelta(days=7)

#         # Find the actual start_filling_time using the helper
#         # We need to pass the allow_weekend_work_in_helper to the helper function
#         start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date,
#                                                      start_of_month, item_number, batch_name, "Filling",
#                                                      allow_weekend_work_in_helper)

#         if start_filling_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Filling.")
#             continue # Cannot schedule this batch

#         # Final check for 7-day holding limit AFTER scheduling
#         if start_filling_time > max_allowed_filling_start_from_qc:
#             print(f"Warning: Batch {item_number} - {batch_name} will exceed 7-day holding limit. Cannot schedule.")
#             continue # Skip if it violates the hard 7-day limit.


#         end_filling_time = start_filling_time + timedelta(hours=filling_runtime)

#         # Update machine and tank availability
#         filling_machine_available_at = end_filling_time
#         # Tank is free when batch leaves for filling
#         holding_tank_free_at[tank_index] = end_filling_time

#         # Update first batch flag
#         if is_first_batch:
#             is_first_batch = False

#         # --- Record Events ---
#         # Create a base event dictionary with NaNs for all new columns
#         # This ensures all columns are present in each event dict before appending
#         base_event = {
#             'Timestamp': None,
#             'Start Mixing Item Number': np.nan, 'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan, 'End Mixing Batch': np.nan,
#             'Mixing Quality Check Item Number': np.nan, 'Mixing Quality Check Batch': np.nan,
#             'Holding Tank 1 Item Number': np.nan, 'Holding Tank 1 Batch': np.nan,
#             'Holding Tank 2 Item Number': np.nan, 'Holding Tank 2 Batch': np.nan,
#             'Holding Tank 3 Item Number': np.nan, 'Holding Tank 3 Batch': np.nan,
#             'Start Filling Item Number': np.nan, 'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan, 'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         }

#         # Mixing events
#         if not is_bulk_for_mixing:
#             mix_start_event = base_event.copy()
#             mix_start_event.update({
#                 'Timestamp': start_mixing_time,
#                 'Start Mixing Item Number': item_number,
#                 'Start Mixing Batch': batch_name
#             })
#             events.append(mix_start_event)

#             mix_end_event = base_event.copy()
#             mix_end_event.update({
#                 'Timestamp': end_mixing_time,
#                 'End Mixing Item Number': item_number,
#                 'End Mixing Batch': batch_name
#             })
#             events.append(mix_end_event)

#         # Quality Check event (always recorded after mixing, even if mixing was skipped for bulk)
#         qc_event = base_event.copy()
#         qc_event.update({
#             'Timestamp': end_qc_time, # QC "completes" at this time
#             'Mixing Quality Check Item Number': item_number,
#             'Mixing Quality Check Batch': batch_name
#         })
#         events.append(qc_event)

#         # Holding Tank Entry event (record which tank it entered)
#         # This will show the batch entering the specific tank.
#         tank_entry_event = base_event.copy()
#         tank_entry_event.update({
#             'Timestamp': tank_entry_time,
#             f'Holding Tank {tank_index + 1} Item Number': item_number,
#             f'Holding Tank {tank_index + 1} Batch': batch_name
#         })
#         events.append(tank_entry_event)

#         # Filling events
#         if not is_bulk_in_name_for_filling:
#             fill_start_event = base_event.copy()
#             fill_start_event.update({
#                 'Timestamp': start_filling_time,
#                 'Start Filling Item Number': item_number,
#                 'Start Filling Batch': batch_name
#             })
#             events.append(fill_start_event)

#             fill_end_event = base_event.copy()
#             fill_end_event.update({
#                 'Timestamp': end_filling_time,
#                 'End Filling Item Number': item_number,
#                 'End Filling Batch': batch_name
#             })
#             events.append(fill_end_event)

#     # Create the final DataFrame
#     df_schedule = pd.DataFrame(events)
#     df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

#     return df_schedule

#### Testing

In [188]:
# # --- Run the scheduler for July 2025 ---
# dfSched = dfUtil.copy()

# overtime = True

# print(f"\n--- Scheduling with Overtime {overtime} ---")
# df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, "2 Shifts")


# #data_table.DataTable(df_schedule_overtime)
# df_schedule_overtime.head(30)

### WIP Integration Test 2

#### Function

In [189]:


# def schedule_liquid_line(df_batches, month_year, overtime=False, feasible=True, shiftNo='2 Shifts'):
#     """
#     Schedules batches for a liquid production line based on given constraints.

#     Args:
#         df_batches (pd.DataFrame): DataFrame with batch details including:
#             - 'Item Number': Batch's code
#             - 'Batch': Name of the product
#             - 'Total Mixing Runtime': Time required for mixing
#             - 'Total Filling Runtime': Time required for filling
#             - 'Due Date': Due date for the batch
#             - 'Bulk' (optional, bool): If True, indicates a bulk item not requiring mixing columns.
#         month_year (str): The month and year for scheduling (e.g., '2024-07').
#         overtime (bool): If True, allows scheduling on weekends (only applicable if shiftNo is not '3 Shifts').
#                          If shiftNo is '3 Shifts', overtime is implicitly True for weekends.
#         feasible (bool): If False, prints a message indicating PPIC MS cannot be met.
#         shiftNo (str): Defines the daily working shifts:
#                        '3 Shifts': 24/7 operation.
#                        '2 Shifts': 7:15 AM - 11:45 PM.

#     Returns:
#         pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
#     """

#     # --- Configuration ---
#     # Define working hours based on shiftNo
#     if shiftNo == '3 Shifts':
#         DAILY_WORKING_HOURS = 24.0
#         DAILY_START_TIME = timedelta(hours=0, minutes=0)
#         DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
#         allow_weekend_work_in_helper = True # 3 Shifts implies 24/7, including weekends
#     else: # Default to '2 Shifts' (or any other value)
#         DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
#         DAILY_START_TIME = timedelta(hours=7, minutes=15)
#         DAILY_END_TIME = timedelta(hours=23, minutes=45)
#         allow_weekend_work_in_helper = overtime # Weekends worked only if 'overtime' flag is True

#     # PPIC Feasibility for current month
#     if not feasible:
#         print("Optimally, PPIC MS cannot be met, feasibility: False\n")

#     # Convert Due Date to datetime objects
#     df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

#     # Initialize scheduling variables
#     # Start of the scheduling month
#     start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

#     # Initialize machine availability times
#     current_time = start_of_month + DAILY_START_TIME

#     # Ensure current_time is on a weekday if not allowing weekend work initially
#     if not allow_weekend_work_in_helper and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#         current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday
#         current_time = current_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                             minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                             second=0, microsecond=0)


#     mixing_machine_available_at = current_time
#     filling_machine_available_at = current_time

#     # Initialize holding tank availability (list of timestamps when each tank is free)
#     # Each element represents when that specific tank will be free.
#     holding_tank_free_at = [current_time, current_time, current_time] # 3 tanks

#     # List to store all events for the final DataFrame
#     events = []

#     # BUG FIX: is_first_batch must be initialized OUTSIDE the loop
#     is_first_batch = True

#     # --- Helper Function to find next available start time for an operation ---
#     def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date,
#                                 initial_scheduling_start_time, item_number, batch_name,
#                                 operation_type, allow_weekend_work):
#         """
#         Calculates the earliest time an operation can start, respecting shifts and continuous run.
#         """
#         proposed_start_time = current_machine_time

#         # Safeguard: Prevent infinite loop or datbue overflow
#         max_search_days = 365 * 1

#         while True:
#             if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
#                 print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} cannot be met. Aborting.")
#                 return None

#             day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                     second=0, microsecond=0)
#             day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
#                                                   minute=(DAILY_END_TIME.seconds % 3600) // 60,
#                                                   second=0, microsecond=0)

#             # If not allowing weekend work, skip weekends
#             if not allow_weekend_work and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
#                 proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                 proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                   minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                   second=0, microsecond=0)
#                 continue # Re-check the new proposed_start_time

#             # Ensure proposed_start_time is not before the daily start time
#             if proposed_start_time < day_start:
#                 proposed_start_time = day_start

#             potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

#             if potential_end_time <= day_end + timedelta(minutes=1):
#                 if potential_end_time.date() > current_batch_due_date.date():
#                     print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
#                     return None
#                 return proposed_start_time
#             else:
#                 # Operation spills over, move to the start of the next working day
#                 proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
#                     hour=DAILY_START_TIME.seconds // 3600,
#                     minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                     second=0, microsecond=0
#                 )
#                 # Ensure it's not a weekend if not allowing weekend work
#                 if not allow_weekend_work and proposed_start_time.weekday() >= 5:
#                     proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
#                     proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
#                                                                       minute=(DAILY_START_TIME.seconds % 3600) // 60,
#                                                                       second=0, microsecond=0)

#     # --- Schedule Each Batch ---
#     for index, row in df_batches.iterrows():
#         item_number = row['Item Number']
#         batch_name = row['Batch']
#         mixing_runtime = row['Total Mixing Runtime']
#         filling_runtime = row['Total Filling Runtime']
#         due_date = row['Due Date']

#         # Check for bulk status for mixing
#         is_bulk_for_mixing = row.get('Bulk', False) # Defaults to False if 'Bulk' column doesn't exist or is not True

#         # Check for 'bulk' in batch name for filling (case-insensitive)
#         is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()


#         # --- Schedule Mixing ---
#         # Only schedule mixing if it's not a bulk item (based on 'Bulk' column)
#         if not is_bulk_for_mixing:
#             start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
#                                                         start_of_month, item_number, batch_name, "Mixing",
#                                                         allow_weekend_work_in_helper)
#             if start_mixing_time is None:
#                 print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
#                 continue
#             end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
#             # mixing_machine_available_at is updated after QC, as QC follows mixing directly
#         else:
#             # If it's a bulk item for mixing, assume mixing is instantaneous or not applicable
#             # Set times to current machine availability to not block other operations
#             start_mixing_time = mixing_machine_available_at
#             end_mixing_time = mixing_machine_available_at
#             #print(f"Batch {item_number} - {batch_name} marked as bulk, skipping mixing scheduling.")


#         # --- Schedule Quality Check ---
#         # QC starts immediately after mixing ends (or "virtual" end_mixing_time for bulk)
#         QC_RUNTIME = 8 # hours

#         # QC must adhere to shifts and continuous run, so use get_next_available_slot
#         start_qc_time = get_next_available_slot(end_mixing_time, QC_RUNTIME, due_date,
#                                                 start_of_month, item_number, batch_name, "Quality Check",
#                                                 allow_weekend_work_in_helper)

#         if start_qc_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Quality Check.")
#             continue

#         end_qc_time = start_qc_time + timedelta(hours=QC_RUNTIME)

#         # Update mixing_machine_available_at to reflect when the output of the mixing+QC stage is free
#         mixing_machine_available_at = end_qc_time


#         # --- Schedule Holding Tank Entry ---
#         # Find the earliest available holding tank
#         earliest_free_tank_time = min(holding_tank_free_at)
#         tank_index = holding_tank_free_at.index(earliest_free_tank_time) # Get the index of the free tank

#         # Batch enters holding tank after QC AND after the tank is free
#         tank_entry_time = max(end_qc_time, earliest_free_tank_time)

#         # --- Schedule Filling ---
#         # The earliest this batch can start filling is:
#         # 1. When the filling machine is free (filling_machine_available_at)
#         # 2. When the batch has finished QC and entered a tank (tank_entry_time)

#         # The constraint for filling start from the batch's perspective (after QC and tank entry)
#         batch_ready_for_filling_time = tank_entry_time

#         # If it's the first batch, ensure it waits for its own mixing/QC completion
#         if is_first_batch:
#             filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)
#         else:
#             # For subsequent batches, filling machine can work on this batch
#             # as soon as it's ready AND filling machine is free.
#             filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)

#         # Check the 7-day max holding time constraint for filling start
#         max_allowed_filling_start_from_qc = end_qc_time + timedelta(days=7)

#         # Find the actual start_filling_time using the helper
#         start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date,
#                                                      start_of_month, item_number, batch_name, "Filling",
#                                                      allow_weekend_work_in_helper)

#         if start_filling_time is None:
#             print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Filling.")
#             continue # Cannot schedule this batch

#         # Final check for 7-day holding limit AFTER scheduling
#         if start_filling_time > max_allowed_filling_start_from_qc:
#             print(f"Warning: Batch {item_number} - {batch_name} will exceed 7-day holding limit. Cannot schedule.")
#             continue # Skip if it violates the hard 7-day limit.


#         end_filling_time = start_filling_time + timedelta(hours=filling_runtime)

#         # Update machine and tank availability
#         filling_machine_available_at = end_filling_time
#         # Tank is free when batch leaves for filling
#         holding_tank_free_at[tank_index] = end_filling_time

#         # Update first batch flag
#         if is_first_batch:
#             is_first_batch = False

#         # --- Record Events ---
#         # Create a base event dictionary with NaNs for all new columns
#         # This ensures all columns are present in each event dict before appending
#         base_event = {
#             'Timestamp': None,
#             'Start Mixing Item Number': np.nan, 'Start Mixing Batch': np.nan,
#             'End Mixing Item Number': np.nan, 'End Mixing Batch': np.nan,
#             'Mixing Quality Check Item Number': np.nan, 'Mixing Quality Check Batch': np.nan,
#             'End Mixing Quality Check Item Number': np.nan, 'End Mixing Quality Check Batch': np.nan,
#             'Holding Tank 1 Item Number': np.nan, 'Holding Tank 1 Batch': np.nan,
#             'Holding Tank 2 Item Number': np.nan, 'Holding Tank 2 Batch': np.nan,
#             'Holding Tank 3 Item Number': np.nan, 'Holding Tank 3 Batch': np.nan,
#             'Start Filling Item Number': np.nan, 'Start Filling Batch': np.nan,
#             'End Filling Item Number': np.nan, 'End Filling Batch': np.nan,
#             'End Filling Batch Due Date': due_date
#         }

#         # Mixing events
#         if not is_bulk_for_mixing:
#             mix_start_event = base_event.copy()
#             mix_start_event.update({
#                 'Timestamp': start_mixing_time,
#                 'Start Mixing Item Number': item_number,
#                 'Start Mixing Batch': batch_name
#             })
#             events.append(mix_start_event)

#             mix_end_event = base_event.copy()
#             mix_end_event.update({
#                 'Timestamp': end_mixing_time,
#                 'End Mixing Item Number': item_number,
#                 'End Mixing Batch': batch_name
#             })
#             events.append(mix_end_event)

#         # Quality Check event (always recorded after mixing, even if mixing was skipped for bulk)
#         qc_start_event = base_event.copy() # New event for QC start
#         qc_start_event.update({
#             'Timestamp': start_qc_time,
#             'Mixing Quality Check Item Number': item_number,
#             'Mixing Quality Check Batch': batch_name,
#             # No End QC info for start event
#         })
#         events.append(qc_start_event)

#         qc_end_event = base_event.copy() # Event for QC end
#         qc_end_event.update({
#             'Timestamp': end_qc_time,
#             'End Mixing Quality Check Item Number': item_number,
#             'End Mixing Quality Check Batch': batch_name
#         })
#         events.append(qc_end_event)


#         # Holding Tank Entry event (record which tank it entered)
#         # This will show the batch entering the specific tank.
#         tank_entry_event = base_event.copy()
#         tank_entry_event.update({
#             'Timestamp': tank_entry_time,
#             f'Holding Tank {tank_index + 1} Item Number': item_number,
#             f'Holding Tank {tank_index + 1} Batch': batch_name
#         })
#         events.append(tank_entry_event)

#         # Filling events
#         if not is_bulk_in_name_for_filling:
#             fill_start_event = base_event.copy()
#             fill_start_event.update({
#                 'Timestamp': start_filling_time,
#                 'Start Filling Item Number': item_number,
#                 'Start Filling Batch': batch_name
#             })
#             events.append(fill_start_event)

#             fill_end_event = base_event.copy()
#             fill_end_event.update({
#                 'Timestamp': end_filling_time,
#                 'End Filling Item Number': item_number,
#                 'End Filling Batch': batch_name
#             })
#             events.append(fill_end_event)

#     # Create the final DataFrame
#     df_schedule = pd.DataFrame(events)
#     df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

#     return df_schedule


#### Testing

In [190]:
# # --- Run the scheduler for July 2025 ---
# dfSched = dfUtil.copy()

# useOvertime = 'Weekday + Saturday'

# overtime = True

# # 'Weekday', 'Weekday + Saturday', '7 Days'

# df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, "2 Shifts")


# data_table.DataTable(df_schedule_overtime)
# #df_schedule_overtime.head(30)

In [191]:
dfSched = df_schedule_overtime.copy()

# dfSched.to_csv("Month Schedule.csv",index = False)
# files.download("Month Schedule.csv")

# WIP Integration Test 2.5

#### Function

In [192]:


def schedule_liquid_line(df_batches, month_year, overtime=False, feasible=True, shiftNo='2 Shifts'):
    """
    Schedules batches for a liquid production line based on given constraints.

    Args:
        df_batches (pd.DataFrame): DataFrame with batch details including:
            - 'Item Number': Batch's code
            - 'Batch': Name of the product
            - 'Total Mixing Runtime': Time required for mixing
            - 'Total Filling Runtime': Time required for filling
            - 'Due Date': Due date for the batch
            - 'Bulk' (optional, bool): If True, indicates a bulk item not requiring mixing columns.
        month_year (str): The month and year for scheduling (e.g., '2024-07').
        overtime (bool): If True, allows scheduling on weekends (only applicable if shiftNo is not '3 Shifts').
                         If shiftNo is '3 Shifts', overtime is implicitly True for weekends.
        feasible (bool): If False, prints a message indicating PPIC MS cannot be met.
        shiftNo (str): Defines the daily working shifts:
                       '3 Shifts': 24/7 operation.
                       '2 Shifts': 7:15 AM - 11:45 PM.

    Returns:
        pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
    """

    # --- Configuration ---
    # Define working hours based on shiftNo
    if shiftNo == '3 Shifts':
        DAILY_WORKING_HOURS = 24.0
        DAILY_START_TIME = timedelta(hours=0, minutes=0)
        DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
        allow_weekend_work_in_helper = True # 3 Shifts implies 24/7, including weekends
    else: # Default to '2 Shifts' (or any other value)
        DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
        DAILY_START_TIME = timedelta(hours=7, minutes=15)
        DAILY_END_TIME = timedelta(hours=23, minutes=45)
        allow_weekend_work_in_helper = overtime # Weekends worked only if 'overtime' flag is True

    # PPIC Feasibility for current month
    if not feasible:
        print("Optimally, PPIC MS cannot be met, feasibility: False\n")

    # Convert Due Date to datetime objects
    df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

    # Initialize scheduling variables
    # Start of the scheduling month
    start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

    # Initialize machine availability times
    current_time = start_of_month + DAILY_START_TIME

    # Ensure current_time is on a weekday if not allowing weekend work initially
    if not allow_weekend_work_in_helper and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
        current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday
        current_time = current_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                            minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                            second=0, microsecond=0)


    mixing_machine_available_at = current_time
    filling_machine_available_at = current_time

    # Initialize holding tank availability (list of timestamps when each tank is free)
    # Each element represents when that specific tank will be free.
    holding_tank_free_at = [current_time, current_time, current_time] # 3 tanks

    # List to store all events for the final DataFrame
    events = []

    # BUG FIX: is_first_batch must be initialized OUTSIDE the loop
    is_first_batch = True

    # --- Helper Function to find next available start time for an operation ---
    def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date,
                                initial_scheduling_start_time, item_number, batch_name,
                                operation_type, allow_weekend_work):
        """
        Calculates the earliest time an operation can start, respecting shifts and continuous run.
        """
        proposed_start_time = current_machine_time

        # Safeguard: Prevent infinite loop or datbue overflow
        max_search_days = 365 * 1

        while True:
            if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
                print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} cannot be met within {shiftNo}. Aborting.")
                return None

            day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                    second=0, microsecond=0)
            day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
                                                  minute=(DAILY_END_TIME.seconds % 3600) // 60,
                                                  second=0, microsecond=0)

            # If not allowing weekend work, skip weekends
            if not allow_weekend_work and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
                proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                  minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                  second=0, microsecond=0)
                continue # Re-check the new proposed_start_time

            # Ensure proposed_start_time is not before the daily start time
            if proposed_start_time < day_start:
                proposed_start_time = day_start

            potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

            if potential_end_time <= day_end + timedelta(minutes=1):
                if potential_end_time.date() > current_batch_due_date.date():
                    print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
                    return None
                return proposed_start_time
            else:
                # Operation spills over, move to the start of the next working day
                proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
                    hour=DAILY_START_TIME.seconds // 3600,
                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                    second=0, microsecond=0
                )
                # Ensure it's not a weekend if not allowing weekend work
                if not allow_weekend_work and proposed_start_time.weekday() >= 5:
                    proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                    proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                      minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                      second=0, microsecond=0)

    # --- Schedule Each Batch ---
    for index, row in df_batches.iterrows():
        item_number = row['Item Number']
        batch_name = row['Batch']
        mixing_runtime = row['Total Mixing Runtime']
        filling_runtime = row['Total Filling Runtime']
        due_date = row['Due Date']

        # Check for bulk status for mixing
        #is_bulk_for_mixing = row.get('Bulk', False) # Defaults to False if 'Bulk' column doesn't exist or is not True
        #is_bulk_for_mixing = str(row.get('Bulk', '')).lower() == 'bulk'

        # Check for 'bulk' in batch name for filling (case-insensitive)
        is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()


        # --- Schedule Mixing ---
        # # Only schedule mixing if it's not a bulk item (based on 'Bulk' column)
        # if not is_bulk_for_mixing:
        #     start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
        #                                                 start_of_month, item_number, batch_name, "Mixing",
        #                                                 allow_weekend_work_in_helper)
        #     if start_mixing_time is None:
        #         print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
        #         continue
        #     end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
        #     # mixing_machine_available_at is updated after QC, as QC follows mixing directly
        # else:
        #     # If it's a bulk item for mixing, assume mixing is instantaneous or not applicable
        #     # Set times to current machine availability to not block other operations
        #     start_mixing_time = mixing_machine_available_at
        #     end_mixing_time = mixing_machine_available_at
        #     #print(f"Batch {item_number} - {batch_name} marked as bulk, skipping mixing scheduling.")

        is_b_batch = str(row['Item Number']).strip().upper().startswith('B-')
        is_a_bulk = row.get('Bulk', False)
        requires_mixing = (is_a_bulk != 'True') or is_b_batch
        is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()

        if requires_mixing:
            start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
                                                        start_of_month, item_number, batch_name, "Mixing",
                                                        allow_weekend_work_in_helper)
            if start_mixing_time is None:
                print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
                continue
            end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
        else:
            start_mixing_time = mixing_machine_available_at
            end_mixing_time = mixing_machine_available_at


        # --- Schedule Quality Check ---
        # QC starts immediately after mixing ends (or "virtual" end_mixing_time for bulk)
        QC_RUNTIME = 8 # hours

        # QC must adhere to shifts and continuous run, so use get_next_available_slot
        start_qc_time = get_next_available_slot(end_mixing_time, QC_RUNTIME, due_date,
                                                start_of_month, item_number, batch_name, "Quality Check",
                                                allow_weekend_work_in_helper)

        if start_qc_time is None:
            print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Quality Check.")
            continue

        end_qc_time = start_qc_time + timedelta(hours=QC_RUNTIME)

        # Update mixing_machine_available_at to reflect when the output of the mixing+QC stage is free
        mixing_machine_available_at = end_qc_time


        # --- Schedule Holding Tank Entry ---
        # Find the earliest available holding tank
        earliest_free_tank_time = min(holding_tank_free_at)
        tank_index = holding_tank_free_at.index(earliest_free_tank_time) # Get the index of the free tank

        # Batch enters holding tank after QC AND after the tank is free
        tank_entry_time = max(end_qc_time, earliest_free_tank_time)

        # --- Schedule Filling ---
        # The earliest this batch can start filling is:
        # 1. When the filling machine is free (filling_machine_available_at)
        # 2. When the batch has finished QC and entered a tank (tank_entry_time)

        # The constraint for filling start from the batch's perspective (after QC and tank entry)
        batch_ready_for_filling_time = tank_entry_time

        # If it's the first batch, ensure it waits for its own mixing/QC completion
        if is_first_batch:
            filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)
        else:
            # For subsequent batches, filling machine can work on this batch
            # as soon as it's ready AND filling machine is free.
            filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)

        # Check the 7-day max holding time constraint for filling start
        max_allowed_filling_start_from_qc = end_qc_time + timedelta(days=7)

        # Find the actual start_filling_time using the helper
        start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date,
                                                     start_of_month, item_number, batch_name, "Filling",
                                                     allow_weekend_work_in_helper)

        if start_filling_time is None:
            print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Filling.")
            continue # Cannot schedule this batch

        # Final check for 7-day holding limit AFTER scheduling
        if start_filling_time > max_allowed_filling_start_from_qc:
            print(f"Warning: Batch {item_number} - {batch_name} will exceed 7-day holding limit. Cannot schedule.")
            continue # Skip if it violates the hard 7-day limit.


        end_filling_time = start_filling_time + timedelta(hours=filling_runtime)

        # Update machine and tank availability
        filling_machine_available_at = end_filling_time
        # Tank is free when batch leaves for filling
        holding_tank_free_at[tank_index] = end_filling_time

        # Update first batch flag
        if is_first_batch:
            is_first_batch = False

        # --- Record Events ---
        # Create a base event dictionary with NaNs for all new columns
        # This ensures all columns are present in each event dict before appending
        base_event = {
            'Timestamp': None,
            'Start Mixing Item Number': np.nan, 'Start Mixing Batch': np.nan,
            'End Mixing Item Number': np.nan, 'End Mixing Batch': np.nan,
            'Mixing Quality Check Item Number': np.nan, 'Mixing Quality Check Batch': np.nan,
            'End Mixing Quality Check Item Number': np.nan, 'End Mixing Quality Check Batch': np.nan,
            'Holding Tank 1 Item Number': np.nan, 'Holding Tank 1 Batch': np.nan,
            'Holding Tank 2 Item Number': np.nan, 'Holding Tank 2 Batch': np.nan,
            'Holding Tank 3 Item Number': np.nan, 'Holding Tank 3 Batch': np.nan,
            'Start Filling Item Number': np.nan, 'Start Filling Batch': np.nan,
            'End Filling Item Number': np.nan, 'End Filling Batch': np.nan,
            'End Filling Batch Due Date': due_date
        }

        # Mixing events
        if requires_mixing:
            mix_start_event = base_event.copy()
            mix_start_event.update({
                'Timestamp': start_mixing_time,
                'Start Mixing Item Number': item_number,
                'Start Mixing Batch': batch_name
            })
            events.append(mix_start_event)

            mix_end_event = base_event.copy()
            mix_end_event.update({
                'Timestamp': end_mixing_time,
                'End Mixing Item Number': item_number,
                'End Mixing Batch': batch_name
            })
            events.append(mix_end_event)

        # Quality Check event (always recorded after mixing, even if mixing was skipped for bulk)
        qc_start_event = base_event.copy() # New event for QC start
        qc_start_event.update({
            'Timestamp': start_qc_time,
            'Mixing Quality Check Item Number': item_number,
            'Mixing Quality Check Batch': batch_name,
            # No End QC info for start event
        })
        events.append(qc_start_event)

        qc_end_event = base_event.copy() # Event for QC end
        qc_end_event.update({
            'Timestamp': end_qc_time,
            'End Mixing Quality Check Item Number': item_number,
            'End Mixing Quality Check Batch': batch_name
        })
        events.append(qc_end_event)


        # Holding Tank Entry event (record which tank it entered)
        # This will show the batch entering the specific tank.
        tank_entry_event = base_event.copy()
        tank_entry_event.update({
            'Timestamp': tank_entry_time,
            f'Holding Tank {tank_index + 1} Item Number': item_number,
            f'Holding Tank {tank_index + 1} Batch': batch_name
        })
        events.append(tank_entry_event)

        # Filling events
        if not is_bulk_in_name_for_filling:
            fill_start_event = base_event.copy()
            fill_start_event.update({
                'Timestamp': start_filling_time,
                'Start Filling Item Number': item_number,
                'Start Filling Batch': batch_name
            })
            events.append(fill_start_event)

            fill_end_event = base_event.copy()
            fill_end_event.update({
                'Timestamp': end_filling_time,
                'End Filling Item Number': item_number,
                'End Filling Batch': batch_name
            })
            events.append(fill_end_event)

    # Create the final DataFrame
    df_schedule = pd.DataFrame(events)
    df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

    return df_schedule


#### Testing

In [193]:
# --- Run the scheduler for July 2025 ---
dfSched = dfUtil.copy()

useOvertime = 'Weekday + Saturday'

overtime = True

shiftNo = '2 Shifts'

if max(max(dfSched['Total Mixing Runtime']), max(dfSched['Total Filling Runtime'])) > 16.5:
  print('Note: Using 3 shifts; needed for a batch running for more than two shift')
  shiftNo = '3 Shifts'


# 'Weekday', 'Weekday + Saturday', '7 Days'

df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, shiftNo)


data_table.DataTable(df_schedule_overtime)
#df_schedule_overtime.head(30)

Note: Using 3 shifts; needed for a batch running for more than two shift
Error: Scheduling for Filling of batch A-14593-00 - RHINOS NEO DROPS 10 ML cannot be met within 3 Shifts. Aborting.
Skipping batch A-14593-00 - RHINOS NEO DROPS 10 ML due to scheduling issues for Filling.


,Timestamp,Start Mixing Item Number,Start Mixing Batch,End Mixing Item Number,End Mixing Batch,Mixing Quality Check Item Number,Mixing Quality Check Batch,End Mixing Quality Check Item Number,End Mixing Quality Check Batch,Holding Tank 1 Item Number,Holding Tank 1 Batch,Holding Tank 2 Item Number,Holding Tank 2 Batch,Holding Tank 3 Item Number,Holding Tank 3 Batch,Start Filling Item Number,Start Filling Batch,End Filling Item Number,End Filling Batch,End Filling Batch Due Date
0,2025-07-01 00:00:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
1,2025-07-01 02:00:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
2,2025-07-01 02:00:00.000000,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
3,2025-07-01 10:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
4,2025-07-01 10:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
5,2025-07-01 10:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,NaN,NaN,2025-07-30
6,2025-07-01 10:00:00.000000,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
7,2025-07-01 13:00:00.000000,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
8,2025-07-01 13:00:00.000000,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML (2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-07-30
9,2025-07-01 20:31:51.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A-12312-00,LACTULOSE SYRUP 3.3 G/ 5ML 60 ML,2025-07-30


In [194]:
dfSched = df_schedule_overtime.copy()

dfSched.to_csv("Month Schedule.csv",index = False)
files.download("Month Schedule.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### WIP Integration Test 3

#### Function

In [ ]:


def schedule_liquid_line(df_batches, month_year, overtime=False, feasible=True, shiftNo='2 Shifts'):
    """
    Schedules batches for a liquid production line based on given constraints.

    Args:
        df_batches (pd.DataFrame): DataFrame with batch details including:
            - 'Item Number': Batch's code
            - 'Batch': Name of the product
            - 'Total Mixing Runtime': Time required for mixing
            - 'Total Filling Runtime': Time required for filling
            - 'Due Date': Due date for the batch
            - 'Bulk' (optional, bool): If True, indicates a bulk item not requiring mixing columns.
        month_year (str): The month and year for scheduling (e.g., '2024-07').
        overtime (bool): If True, allows scheduling on weekends (only applicable if shiftNo is not '3 Shifts').
                         If shiftNo is '3 Shifts', overtime is implicitly True for weekends.
        feasible (bool): If False, prints a message indicating PPIC MS cannot be met.
        shiftNo (str): Defines the daily working shifts:
                       '3 Shifts': 24/7 operation.
                       '2 Shifts': 7:15 AM - 11:45 PM.

    Returns:
        pd.DataFrame: Scheduling DataFrame with timestamps and batch events.
    """

    # --- Configuration ---
    # Define working hours based on shiftNo
    if shiftNo == '3 Shifts':
        DAILY_WORKING_HOURS = 24.0
        DAILY_START_TIME = timedelta(hours=0, minutes=0)
        DAILY_END_TIME = timedelta(hours=23, minutes=59) # Effectively end of day
        allow_weekend_work_in_helper = True # 3 Shifts implies 24/7, including weekends
    else: # Default to '2 Shifts' (or any other value)
        DAILY_WORKING_HOURS = 16.5 # 16 hours 30 minutes
        DAILY_START_TIME = timedelta(hours=7, minutes=15)
        DAILY_END_TIME = timedelta(hours=23, minutes=45)
        allow_weekend_work_in_helper = overtime # Weekends worked only if 'overtime' flag is True

    # PPIC Feasibility for current month
    if not feasible:
        print("Optimally, PPIC MS cannot be met, feasibility: False\n")

    # Convert Due Date to datetime objects
    df_batches['Due Date'] = pd.to_datetime(df_batches['Due Date'])

    # Initialize scheduling variables
    # Start of the scheduling month
    start_of_month = datetime.strptime(f"{month_year}-01", "%Y-%m-%d")

    # Initialize machine availability times
    current_time = start_of_month + DAILY_START_TIME

    # Ensure current_time is on a weekday if not allowing weekend work initially
    if not allow_weekend_work_in_helper and current_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
        current_time = current_time + timedelta(days=(7 - current_time.weekday())) # Advance to Monday
        current_time = current_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                            minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                            second=0, microsecond=0)


    mixing_machine_available_at = current_time
    filling_machine_available_at = current_time

    # Initialize holding tank availability (list of timestamps when each tank is free)
    # Each element represents when that specific tank will be free.
    holding_tank_free_at = [current_time, current_time, current_time] # 3 tanks

    # List to store all events for the final DataFrame
    events = []

    # BUG FIX: is_first_batch must be initialized OUTSIDE the loop
    is_first_batch = True

    # --- Helper Function to find next available start time for an operation ---
    def get_next_available_slot(current_machine_time, duration_hours, current_batch_due_date,
                                initial_scheduling_start_time, item_number, batch_name,
                                operation_type, allow_weekend_work):
        """
        Calculates the earliest time an operation can start, respecting shifts and continuous run.
        """
        proposed_start_time = current_machine_time

        # Safeguard: Prevent infinite loop or date overflow
        max_search_days = 365 * 5

        while True:
            if (proposed_start_time - initial_scheduling_start_time).days > max_search_days:
                print(f"Error: Scheduling for {operation_type} of batch {item_number} - {batch_name} exceeded max search duration ({max_search_days} days). Aborting.")
                return None

            day_start = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                    second=0, microsecond=0)
            day_end = proposed_start_time.replace(hour=DAILY_END_TIME.seconds // 3600,
                                                  minute=(DAILY_END_TIME.seconds % 3600) // 60,
                                                  second=0, microsecond=0)

            # If not allowing weekend work, skip weekends
            if not allow_weekend_work and proposed_start_time.weekday() >= 5: # 5 is Saturday, 6 is Sunday
                proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                  minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                  second=0, microsecond=0)
                continue # Re-check the new proposed_start_time

            # Ensure proposed_start_time is not before the daily start time
            if proposed_start_time < day_start:
                proposed_start_time = day_start

            potential_end_time = proposed_start_time + timedelta(hours=duration_hours)

            if potential_end_time <= day_end + timedelta(minutes=1):
                if potential_end_time.date() > current_batch_due_date.date():
                    print(f"Warning: Batch {item_number} - {batch_name} ({operation_type}) cannot be completed by due date {current_batch_due_date.date()}.")
                    return None
                return proposed_start_time
            else:
                # Operation spills over, move to the start of the next working day
                proposed_start_time = (proposed_start_time + timedelta(days=1)).replace(
                    hour=DAILY_START_TIME.seconds // 3600,
                    minute=(DAILY_START_TIME.seconds % 3600) // 60,
                    second=0, microsecond=0
                )
                # Ensure it's not a weekend if not allowing weekend work
                if not allow_weekend_work and proposed_start_time.weekday() >= 5:
                    proposed_start_time = proposed_start_time + timedelta(days=(7 - proposed_start_time.weekday()))
                    proposed_start_time = proposed_start_time.replace(hour=DAILY_START_TIME.seconds // 3600,
                                                                      minute=(DAILY_START_TIME.seconds % 3600) // 60,
                                                                      second=0, microsecond=0)

    # --- Schedule Each Batch ---
    for index, row in df_batches.iterrows():
        item_number = row['Item Number']
        batch_name = row['Batch']
        mixing_runtime = row['Total Mixing Runtime']
        filling_runtime = row['Total Filling Runtime']
        due_date = row['Due Date']

        # Check for bulk status for mixing
        is_bulk_for_mixing = row.get('Bulk', False) # Defaults to False if 'Bulk' column doesn't exist or is not True

        # Check for 'bulk' in batch name for filling (case-insensitive)
        is_bulk_in_name_for_filling = 'bulk' in str(batch_name).lower()


        # --- Schedule Mixing ---
        # Only schedule mixing if it's not a bulk item (based on 'Bulk' column)
        if not is_bulk_for_mixing:
            start_mixing_time = get_next_available_slot(mixing_machine_available_at, mixing_runtime, due_date,
                                                        start_of_month, item_number, batch_name, "Mixing",
                                                        allow_weekend_work_in_helper)
            if start_mixing_time is None:
                print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Mixing.")
                continue
            end_mixing_time = start_mixing_time + timedelta(hours=mixing_runtime)
            # mixing_machine_available_at is updated after QC, as QC follows mixing directly
        else:
            # If it's a bulk item for mixing, assume mixing is instantaneous or not applicable
            # Set times to current machine availability to not block other operations
            start_mixing_time = mixing_machine_available_at
            end_mixing_time = mixing_machine_available_at
            #print(f"Batch {item_number} - {batch_name} marked as bulk, skipping mixing scheduling.")


        # --- Schedule Quality Check ---
        # QC starts immediately after mixing ends (or "virtual" end_mixing_time for bulk)
        QC_RUNTIME = 8 # hours

        # QC must adhere to shifts and continuous run, so use get_next_available_slot
        start_qc_time = get_next_available_slot(end_mixing_time, QC_RUNTIME, due_date,
                                                start_of_month, item_number, batch_name, "Quality Check",
                                                allow_weekend_work_in_helper)

        if start_qc_time is None:
            print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Quality Check.")
            continue

        end_qc_time = start_qc_time + timedelta(hours=QC_RUNTIME)

        # Update mixing_machine_available_at to reflect when the output of the mixing+QC stage is free
        mixing_machine_available_at = end_qc_time


        # --- Schedule Holding Tank Entry ---
        # Find the earliest available holding tank
        earliest_free_tank_time = min(holding_tank_free_at)
        tank_index = holding_tank_free_at.index(earliest_free_tank_time) # Get the index of the free tank

        # Batch enters holding tank after QC AND after the tank is free
        tank_entry_time = max(end_qc_time, earliest_free_tank_time)

        # --- Schedule Filling ---
        # The earliest this batch can start filling is:
        # 1. When the filling machine is free (filling_machine_available_at)
        # 2. When the batch has finished QC and entered a tank (tank_entry_time)

        # The constraint for filling start from the batch's perspective (after QC and tank entry)
        batch_ready_for_filling_time = tank_entry_time

        # If it's the first batch, ensure it waits for its own mixing/QC completion
        if is_first_batch:
            filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)
        else:
            # For subsequent batches, filling machine can work on this batch
            # as soon as it's ready AND filling machine is free.
            filling_start_constraint = max(filling_machine_available_at, batch_ready_for_filling_time)

        # Check the 7-day max holding time constraint for filling start
        max_allowed_filling_start_from_qc = end_qc_time + timedelta(days=7)

        # Find the actual start_filling_time using the helper
        start_filling_time = get_next_available_slot(filling_start_constraint, filling_runtime, due_date,
                                                     start_of_month, item_number, batch_name, "Filling",
                                                     allow_weekend_work_in_helper)

        if start_filling_time is None:
            print(f"Skipping batch {item_number} - {batch_name} due to scheduling issues for Filling.")
            continue # Cannot schedule this batch

        # Final check for 7-day holding limit AFTER scheduling
        if start_filling_time > max_allowed_filling_start_from_qc:
            print(f"Warning: Batch {item_number} - {batch_name} will exceed 7-day holding limit. Cannot schedule.")
            continue # Skip if it violates the hard 7-day limit.


        end_filling_time = start_filling_time + timedelta(hours=filling_runtime)

        # Update machine and tank availability
        filling_machine_available_at = end_filling_time
        # Tank is free when batch leaves for filling
        holding_tank_free_at[tank_index] = end_filling_time

        # Update first batch flag
        if is_first_batch:
            is_first_batch = False

        # --- Record Events ---
        # Create a base event dictionary with NaNs for all new columns
        # This ensures all columns are present in each event dict before appending
        base_event = {
            'Timestamp': None,
            'Start Mixing Item Number': np.nan, 'Start Mixing Batch': np.nan,
            'End Mixing Item Number': np.nan, 'End Mixing Batch': np.nan,
            'Mixing Quality Check Item Number': np.nan, 'Mixing Quality Check Batch': np.nan,
            'End Mixing Quality Check Item Number': np.nan, 'End Mixing Quality Check Batch': np.nan,
            'Holding Tank 1 Item Number': np.nan, 'Holding Tank 1 Batch': np.nan,
            'Holding Tank 2 Item Number': np.nan, 'Holding Tank 2 Batch': np.nan,
            'Holding Tank 3 Item Number': np.nan, 'Holding Tank 3 Batch': np.nan,
            'Start Filling Item Number': np.nan, 'Start Filling Batch': np.nan,
            'End Filling Item Number': np.nan, 'End Filling Batch': np.nan,
            'End Filling Batch Due Date': due_date
        }

        # Mixing events
        if not is_bulk_for_mixing:
            mix_start_event = base_event.copy()
            mix_start_event.update({
                'Timestamp': start_mixing_time,
                'Start Mixing Item Number': item_number,
                'Start Mixing Batch': batch_name
            })
            events.append(mix_start_event)

            mix_end_event = base_event.copy()
            mix_end_event.update({
                'Timestamp': end_mixing_time,
                'End Mixing Item Number': item_number,
                'End Mixing Batch': batch_name
            })
            events.append(mix_end_event)

        # Quality Check event (always recorded after mixing, even if mixing was skipped for bulk)
        qc_start_event = base_event.copy() # New event for QC start
        qc_start_event.update({
            'Timestamp': start_qc_time,
            'Mixing Quality Check Item Number': item_number,
            'Mixing Quality Check Batch': batch_name,
            # No End QC info for start event
        })
        events.append(qc_start_event)

        qc_end_event = base_event.copy() # Event for QC end
        qc_end_event.update({
            'Timestamp': end_qc_time,
            'End Mixing Quality Check Item Number': item_number,
            'End Mixing Quality Check Batch': batch_name
        })
        events.append(qc_end_event)


        # Holding Tank Entry event (record which tank it entered)
        # This will show the batch entering the specific tank.
        tank_entry_event = base_event.copy()
        tank_entry_event.update({
            'Timestamp': tank_entry_time,
            f'Holding Tank {tank_index + 1} Item Number': item_number,
            f'Holding Tank {tank_index + 1} Batch': batch_name
        })
        events.append(tank_entry_event)

        # Filling events
        if not is_bulk_in_name_for_filling:
            fill_start_event = base_event.copy()
            fill_start_event.update({
                'Timestamp': start_filling_time,
                'Start Filling Item Number': item_number,
                'Start Filling Batch': batch_name
            })
            events.append(fill_start_event)

            fill_end_event = base_event.copy()
            fill_end_event.update({
                'Timestamp': end_filling_time,
                'End Filling Item Number': item_number,
                'End Filling Batch': batch_name
            })
            events.append(fill_end_event)

    # Create the final DataFrame
    df_schedule = pd.DataFrame(events)
    df_schedule = df_schedule.sort_values(by='Timestamp').reset_index(drop=True)

    return df_schedule


#### Testing

In [ ]:
# --- Run the scheduler for July 2025 ---
dfSched = dfUtil.copy()

overtime = True

print(f"\n--- Scheduling with Overtime {overtime} ---")
df_schedule_overtime = schedule_liquid_line(dfSched, f'{year}-0{list(calendar.month_abbr).index(month)}', overtime, feasible, "2 Shifts")


data_table.DataTable(df_schedule_overtime)
#df_schedule_overtime.head(30)